In [1]:
import pandas as pd
import numpy as np
import xarray as xr
import os

import warnings
warnings.filterwarnings("ignore")

In [23]:
# import raw county-level predictions of meta-random-forest models
# each county has five predictionsfor five dominant soil mapunits, obtain one mean prediction by area-weighted average
model_inputs_dir = 'D:/carbon credit/metaforest/county_project/inputs/'
model_outputs_dir = 'D:/carbon credit/metaforest/county_project/model_2405_lt/'
county_outputs_dir = 'D:/carbon credit/plot/county_map/cost_benefit/2024/05_lt/'
plot_dir = 'D:/carbon credit/plot/county_map/2024/05_lt/'
if not os.path.exists(plot_dir):
    os.mkdir(plot_dir)
if not os.path.exists(county_outputs_dir):
    os.mkdir(county_outputs_dir)
# take area-weighted average on model predictions over five main soil types in each county
model_pred = pd.read_csv(model_outputs_dir + 'metaforest_county_prediction_new.csv')
model_pred_se = pd.read_csv(model_outputs_dir + 'metaforest_county_prediction_se_new.csv')
county_soil = pd.read_csv(model_inputs_dir + 'county_soil_mapunit_ratio.csv')
county_manage = pd.read_csv(model_inputs_dir + 'county_management_inputs.csv')
soilmap_ratio = county_soil['soilmap_ratio']

group_county = pd.Series([i for i in range(3108) for n in range(5)])
print(model_pred.shape)
state = county_soil['state'].groupby(group_county).first().reset_index(drop=True)
FIPS = county_soil['county'].groupby(group_county).first().reset_index(drop=True)
# take area-weighted average over the five predictions for each county
pred_county = model_pred.iloc[:,3:].mul(soilmap_ratio, axis=0).groupby(group_county).sum().reset_index(drop=True)
pred_county_se = model_pred_se.iloc[:,3:].mul(soilmap_ratio, axis=0).groupby(group_county).sum().reset_index(drop=True)

pred_county.insert(0, 'State', state)
pred_county.insert(1, 'FIPS', FIPS)
pred_county_se.insert(0, 'State', state)
pred_county_se.insert(1, 'FIPS', FIPS)

# calcualte predictions for corn-soybean rotation
pred_county['Rot_delta_N2O_cc'] = (pred_county['Rot_delta_N2O_cc_corn'] + pred_county['Rot_delta_N2O_cc_soy']) / 2
pred_county['Rot_delta_N2O_nt'] = (pred_county['Rot_delta_N2O_nt_corn'] + pred_county['Rot_delta_N2O_nt_soy']) / 2
pred_county['Rot_delta_lch_cc'] = (pred_county['Rot_delta_lch_cc_corn'] + pred_county['Rot_delta_lch_cc_soy']) / 2
pred_county['Rot_delta_lch_nt'] = (pred_county['Rot_delta_lch_nt_corn'] + pred_county['Rot_delta_lch_nt_soy']) / 2
pred_county['Rot_delta_yld_cc'] = (pred_county['Rot_delta_yld_cc_corn'] + pred_county['Rot_delta_yld_cc_soy']) / 2
pred_county['Rot_delta_yld_nt'] = (pred_county['Rot_delta_yld_nt_corn'] + pred_county['Rot_delta_yld_nt_soy']) / 2
pred_county_se['Rot_delta_N2O_cc'] = (pred_county_se['Rot_delta_N2O_cc_corn'] + pred_county_se['Rot_delta_N2O_cc_soy']) / 2
pred_county_se['Rot_delta_N2O_nt'] = (pred_county_se['Rot_delta_N2O_nt_corn'] + pred_county_se['Rot_delta_N2O_nt_soy']) / 2
pred_county_se['Rot_delta_lch_cc'] = (pred_county_se['Rot_delta_lch_cc_corn'] + pred_county_se['Rot_delta_lch_cc_soy']) / 2
pred_county_se['Rot_delta_lch_nt'] = (pred_county_se['Rot_delta_lch_nt_corn'] + pred_county_se['Rot_delta_lch_nt_soy']) / 2
pred_county_se['Rot_delta_yld_cc'] = (pred_county_se['Rot_delta_yld_cc_corn'] + pred_county_se['Rot_delta_yld_cc_soy']) / 2
pred_county_se['Rot_delta_yld_nt'] = (pred_county_se['Rot_delta_yld_nt_corn'] + pred_county_se['Rot_delta_yld_nt_soy']) / 2

pred_county = pd.concat((pred_county, county_manage.iloc[:, 2:]), axis = 1)
pred_county_se = pd.concat((pred_county_se, county_manage.iloc[:, 2:]), axis = 1)
# pred_county = pred_county.replace(0, np.nan)
# pred_county_se = pred_county_se.replace(0, np.nan)

pred_county.to_csv(county_outputs_dir + 'county_pred.csv')
pred_county_se.to_csv(county_outputs_dir + 'county_pred_se.csv')

(15540, 29)


In [24]:
# aggregate model prediction (/ha) at county level with real corn-soybean production area

def pred_aggregate(pred_county, file_name):
    
#     vars = ['Rot_delta_SOC_30', 'Rot_delta_SOC_60', 'Rot_delta_N2O_cc_corn', 'Rot_delta_N2O_cc_soy',
#             'Rot_delta_N2O_nt_corn', 'Rot_delta_N2O_nt_soy', 'Rot_delta_lch_cc_corn', 'Rot_delta_lch_cc_soy',
#             'Rot_delta_lch_nt_corn', 'Rot_delta_lch_nt_soy', 'Rot_delta_yld_cc_corn', 'Rot_delta_yld_cc_soy',
#             'Rot_delta_yld_nt_corn', 'Rot_delta_yld_nt_soy',
#             'Con_delta_SOC_30', 'Con_delta_SOC_60', 'Con_delta_N2O_cc', 'Con_delta_N2O_nt', 
#             'Con_delta_lch_cc', 'Con_delta_lch_nt', 'Con_delta_yld_cc', 'Con_delta_yld_nt']

    # aggregate predicted densities at county level
    sum_county = pd.DataFrame(index=range(pred_county.shape[0]))

    corn_area = pred_county['corn_area'] * 0.404686
    crop_area = pred_county['crop_area'] * 0.404686
    cc_area = pred_county['cc_adopt'] * 0.404686
    ct_area = pred_county['ct_adopt'] * 0.404686
    nt_area = pred_county['nt_adopt'] * 0.404686
    cc_ratio = cc_area / crop_area
    ct_ratio = ct_area / crop_area
    nt_ratio = nt_area / crop_area
    rot_ratio = pred_county['rotation_ratio']
    area_rot_cc = corn_area * cc_ratio * rot_ratio
    area_rot_nt = corn_area * nt_ratio * rot_ratio
    area_con_cc = corn_area * cc_ratio * (1-rot_ratio)
    area_con_nt = corn_area * nt_ratio * (1-rot_ratio)

    sum_county['crop_area'] = crop_area
    sum_county['corn_area'] = corn_area
    sum_county['cc_ratio'] = cc_ratio
    sum_county['nt_ratio'] = nt_ratio
    sum_county['rot_ratio'] = rot_ratio
    sum_county['corn_cc_area'] = corn_area * cc_ratio
    sum_county['corn_ct_area'] = corn_area * ct_ratio
    sum_county['corn_nt_area'] = corn_area * nt_ratio
    
    # aggreagate delta_SOC in corn-soy and corn-corn systems
    sum_county['sum_soc_cc_rot'] = pred_county['Rot_delta_SOC_30'] * area_rot_cc
    sum_county['sum_soc_nt_rot'] = pred_county['Rot_delta_SOC_60'] * area_rot_nt
    sum_county['sum_soc_cc_con'] = pred_county['Con_delta_SOC_30'] * area_con_cc
    sum_county['sum_soc_nt_con'] = pred_county['Con_delta_SOC_60'] * area_con_nt
    
    sum_county['sum_cc_soc'] = sum_county['sum_soc_cc_rot'] + sum_county['sum_soc_cc_con']
    sum_county['sum_nt_soc'] = sum_county['sum_soc_nt_rot'] + sum_county['sum_soc_nt_con']
    # aggreagate delta_N2O
    sum_county['sum_n2o_cc_rot_corn'] = pred_county['Rot_delta_N2O_cc_corn'] * area_rot_cc / 2
    sum_county['sum_n2o_cc_rot_soy'] = pred_county['Rot_delta_N2O_cc_soy'] * area_rot_cc / 2
    sum_county['sum_n2o_nt_rot_corn'] = pred_county['Rot_delta_N2O_nt_corn'] * area_rot_nt / 2
    sum_county['sum_n2o_nt_rot_soy'] = pred_county['Rot_delta_N2O_nt_soy'] * area_rot_nt / 2
    
    sum_county['sum_n2o_cc_rot'] = sum_county['sum_n2o_cc_rot_corn'] + sum_county['sum_n2o_cc_rot_soy']
    sum_county['sum_n2o_nt_rot'] = sum_county['sum_n2o_nt_rot_corn'] + sum_county['sum_n2o_nt_rot_soy']
    sum_county['sum_n2o_cc_con'] = pred_county['Con_delta_N2O_cc'] * area_con_cc
    sum_county['sum_n2o_nt_con'] = pred_county['Con_delta_N2O_nt'] * area_con_nt

    sum_county['sum_cc_n2o'] = sum_county['sum_n2o_cc_rot'] + sum_county['sum_n2o_cc_con']
    sum_county['sum_nt_n2o'] = sum_county['sum_n2o_nt_rot'] + sum_county['sum_n2o_nt_con']
    # aggreagate delta_N leaching
    sum_county['sum_lch_cc_rot_corn'] = pred_county['Rot_delta_lch_cc_corn'] * area_rot_cc / 2
    sum_county['sum_lch_cc_rot_soy'] = pred_county['Rot_delta_lch_cc_soy'] * area_rot_cc / 2
    sum_county['sum_lch_nt_rot_corn'] = pred_county['Rot_delta_lch_nt_corn'] * area_rot_nt / 2
    sum_county['sum_lch_nt_rot_soy'] = pred_county['Rot_delta_lch_nt_soy'] * area_rot_nt / 2
    
    sum_county['sum_lch_cc_rot'] = sum_county['sum_lch_cc_rot_corn'] + sum_county['sum_lch_cc_rot_soy']
    sum_county['sum_lch_nt_rot'] = sum_county['sum_lch_nt_rot_corn'] + sum_county['sum_lch_nt_rot_soy']
    sum_county['sum_lch_cc_con'] = pred_county['Con_delta_lch_cc'] * area_con_cc
    sum_county['sum_lch_nt_con'] = pred_county['Con_delta_lch_nt'] * area_con_nt

    sum_county['sum_cc_lch'] = sum_county['sum_lch_cc_rot'] + sum_county['sum_lch_cc_con']
    sum_county['sum_nt_lch'] = sum_county['sum_lch_nt_rot'] + sum_county['sum_lch_nt_con']
    # aggreagate delta_yield
    sum_county['sum_yld_cc_rot_corn'] = pred_county['Rot_delta_yld_cc_corn'] * area_rot_cc / 2
    sum_county['sum_yld_cc_rot_soy'] = pred_county['Rot_delta_yld_cc_soy'] * area_rot_cc / 2
    sum_county['sum_yld_nt_rot_corn'] = pred_county['Rot_delta_yld_nt_corn'] * area_rot_nt / 2
    sum_county['sum_yld_nt_rot_soy'] = pred_county['Rot_delta_yld_nt_soy'] * area_rot_nt / 2
    
    sum_county['sum_yld_cc_rot'] = sum_county['sum_yld_cc_rot_corn'] + sum_county['sum_yld_cc_rot_soy']
    sum_county['sum_yld_nt_rot'] = sum_county['sum_yld_nt_rot_corn'] + sum_county['sum_yld_nt_rot_soy']
    sum_county['sum_yld_cc_con'] = pred_county['Con_delta_yld_cc'] * area_con_cc
    sum_county['sum_yld_nt_con'] = pred_county['Con_delta_yld_nt'] * area_con_nt

    sum_county['sum_cc_yld'] = sum_county['sum_yld_cc_rot'] + sum_county['sum_yld_cc_con']
    sum_county['sum_nt_yld'] = sum_county['sum_yld_nt_rot'] + sum_county['sum_yld_nt_con']
    
    
    sum_county.insert(0, 'FIPS', pred_county['FIPS'])
    sum_county.to_csv(county_outputs_dir + file_name)
pred_aggregate(pred_county, 'county_aggregates_2017.csv')
pred_aggregate(pred_county_se, 'county_aggregates_2017_se.csv')

In [25]:
# cost-benefit analysis at county level

def cost_benefit_analysis():
    scale = 1
#     sum_county = sum_county.replace(np.nan, 0)

    class cost_bef:
        def __init__(self, farm_op_cc, farm_op_nt, lt_save_cc, lt_save_nt, corn_pr, soy_pr, co2_pr, ind_EF, re_rate):

            # GHG in CO2 eq, county total: sum_soc, sum_leach, sum_n2o, sum_yld in unit 10^3t
            self.rot_ratio = sum_county['rot_ratio']
            self.con_ratio = 1 - sum_county['rot_ratio']
            self.cc_area = sum_county['corn_cc_area']
            self.nt_area = sum_county['corn_nt_area']
            self.cc_GHG_soc = sum_county['sum_cc_soc'] * 44 / 12
            self.cc_GHG_n2o = - sum_county['sum_cc_n2o'] / 28 * 44 * 0.273
            self.cc_GHG_lch = - sum_county['sum_cc_lch'] * ind_EF / 28 * 44 * 0.273
            self.cc_GHG = self.cc_GHG_soc + self.cc_GHG_n2o + self.cc_GHG_lch
            self.nt_GHG_soc = sum_county['sum_nt_soc'] * 44 / 12
            self.nt_GHG_n2o = - sum_county['sum_nt_n2o'] / 28 * 44 * 0.273
            self.nt_GHG_lch = - sum_county['sum_nt_lch'] * ind_EF / 28 * 44 * 0.273
            self.nt_GHG = self.nt_GHG_soc + self.nt_GHG_n2o + self.nt_GHG_lch
            # carbon benefits, in USD $
            self.cc_COB_soc = self.cc_GHG_soc * co2_pr * re_rate
            self.cc_COB_n2o = (self.cc_GHG_n2o + self.cc_GHG_lch) * co2_pr * re_rate
            self.cc_COB = self.cc_COB_soc + self.cc_COB_n2o
            self.nt_COB_soc = self.nt_GHG_soc * co2_pr * re_rate
            self.nt_COB_n2o = (self.nt_GHG_n2o + self.nt_GHG_lch) * co2_pr * re_rate
            self.nt_COB = self.nt_COB_soc + self.nt_COB_n2o

            # on-farm cost, in USD $
            self.cc_yld = (corn_pr * sum_county['sum_yld_cc_con'] + corn_pr * sum_county['sum_yld_cc_rot_corn'] \
                           + soy_pr * sum_county['sum_yld_cc_rot_soy'])
            self.cc_opt = farm_op_cc * sum_county['corn_cc_area']
            self.cc_sav = lt_save_cc * sum_county['corn_cc_area']
            self.cc_FC = self.cc_yld + self.cc_opt + self.cc_sav
            self.nt_yld = (corn_pr * sum_county['sum_yld_nt_con'] + corn_pr * sum_county['sum_yld_nt_rot_corn'] \
                           + soy_pr * sum_county['sum_yld_nt_rot_soy'])
            self.nt_opt = farm_op_nt * sum_county['corn_nt_area']
            self.nt_sav = lt_save_nt * sum_county['corn_nt_area']
            self.nt_FC = self.nt_yld + self.nt_opt + self.nt_sav

            # Social benefit, carbon benefits + yield return
            self.cc_SB = self.cc_COB * re_rate + self.cc_yld
            self.nt_SB = self.nt_COB * re_rate + self.nt_yld
            # Total return to farmer
            self.cc_MRG = self.cc_COB * re_rate + self.cc_FC
            self.nt_MRG = self.nt_COB * re_rate + self.nt_FC

            # CO2 mitigation intensity and efficiency at $/ha
            self.cc_intensity = self.cc_GHG / self.cc_area
            self.nt_intensity = self.nt_GHG / self.nt_area
            self.cc_cost = self.cc_FC / self.cc_area
            self.nt_cost = self.nt_FC / self.nt_area
            self.cc_efficiency = self.cc_FC / self.cc_GHG
            self.nt_efficiency = self.nt_FC / self.nt_GHG

            self.cc_intensity_us = self.cc_GHG.sum()  / sum_county['corn_cc_area'].sum()
            self.nt_intensity_us = self.nt_GHG.sum()  / sum_county['corn_nt_area'].sum()
            self.cc_efficiency_us = (self.cc_FC).sum() / (self.cc_GHG).sum()
            self.nt_efficiency_us = (self.nt_FC).sum() / (self.nt_GHG).sum()
            print(self.cc_intensity_us, self.nt_intensity_us, self.cc_efficiency_us, self.nt_efficiency_us)


    # set prices for farm_op_cc, farm_op_nt, lt_save_cc, lt_save_nt, corn_pr, soy_pr, co2_pr, ind_EF, re_rate
    margin = cost_bef(-98, 57, 32, -29, 143, 359, 46, 0.011, 0.6) # mean margin
    margin_max = cost_bef(-40, 95, 42, -12, 136, 314, 167, 0.011, 0.6) # max margin
    margin_min = cost_bef(-206, 36, 26, -39, 170, 468, 27, 0.011, 0.6) # min margin

    print(margin.cc_COB.max(), margin.nt_COB.max(), margin.cc_FC.max(), margin.nt_FC.max(), 
          margin.cc_MRG.max(), margin.nt_MRG.max())
    est_cty_cc = pd.concat((margin.cc_COB_soc, margin.cc_COB_n2o, margin.cc_COB, margin.cc_yld,
                            margin.cc_opt, margin.cc_sav, margin.cc_FC, margin.cc_SB, margin.cc_MRG), axis = 1)
    est_cty_nt = pd.concat((margin.nt_COB_soc, margin.nt_COB_n2o, margin.nt_COB, margin.nt_yld,
                            margin.nt_opt, margin.nt_sav, margin.nt_FC, margin.nt_SB, margin.nt_MRG), axis = 1)
    est_all = pd.concat((est_cty_cc / scale, est_cty_nt / scale, margin.cc_intensity, margin.nt_intensity, margin.cc_cost,
                         margin.nt_cost, margin.cc_efficiency, margin.nt_efficiency), axis = 1)
    
    est_cty_cc_mx = pd.concat((margin_max.cc_COB_soc, margin_max.cc_COB_n2o, margin_max.cc_COB, margin_max.cc_yld,
                            margin_max.cc_opt, margin_max.cc_sav, margin_max.cc_FC, margin_max.cc_SB, margin_max.cc_MRG), axis = 1)
    est_cty_nt_mx = pd.concat((margin_max.nt_COB_soc, margin_max.nt_COB_n2o, margin_max.nt_COB, margin_max.nt_yld,
                            margin_max.nt_opt, margin_max.nt_sav, margin_max.nt_FC, margin_max.nt_SB, margin_max.nt_MRG), axis = 1)
    est_all_mx = pd.concat((est_cty_cc_mx / scale, est_cty_nt_mx / scale, margin_max.cc_intensity, margin_max.nt_intensity, 
                         margin_max.cc_cost, margin_max.nt_cost, margin_max.cc_efficiency, margin_max.nt_efficiency), axis = 1)
    
    est_cty_cc_mn = pd.concat((margin_min.cc_COB_soc, margin_min.cc_COB_n2o, margin_min.cc_COB, margin_min.cc_yld,
                            margin_min.cc_opt, margin_min.cc_sav, margin_min.cc_FC, margin_min.cc_SB, margin_min.cc_MRG), axis = 1)
    est_cty_nt_mn = pd.concat((margin_min.nt_COB_soc, margin_min.nt_COB_n2o, margin_min.nt_COB, margin_min.nt_yld,
                            margin_min.nt_opt, margin_min.nt_sav, margin_min.nt_FC, margin_min.nt_SB, margin_min.nt_MRG), axis = 1)
    est_all_mn = pd.concat((est_cty_cc_mn / scale, est_cty_nt_mn / scale, margin_min.cc_intensity, margin_min.nt_intensity, 
                         margin_min.cc_cost, margin_min.nt_cost, margin_min.cc_efficiency, margin_min.nt_efficiency), axis = 1)
    
    items = ['CB_SOC', 'CB_N2O', 'CB', 'YldReturn', 'Operation', 'Longterm', 'FC', 'SB', 'MRG']
    col_names = [head + item for head in ['CC_','NT_'] for item in items] + ['cc_int', 'nt_int', 'cc_cost', 'nt_cost', 'cc_eff', 'nt_eff']
    est_all.columns = col_names
    est_all_mx.columns = col_names
    est_all_mn.columns = col_names
    est_all['FIPS'] = sum_county['FIPS']
    est_all_mx['FIPS'] = sum_county['FIPS']
    est_all_mn['FIPS'] = sum_county['FIPS']
    est_all.to_csv(county_outputs_dir + 'cost_bef_historic_2017.csv')
    est_all_mx.to_csv(county_outputs_dir + 'cost_bef_historic_2017_max.csv')
    est_all_mn.to_csv(county_outputs_dir + 'cost_bef_historic_2017_min.csv')


0.6764139189218371 2.3333895869372148 -120.14162557785016 -3.8365609765990807
0.6764139189218371 2.3333895869372148 -17.969372264570186 20.75159324955555
0.6764139189218371 2.3333895869372148 -293.7461562923449 -20.4758369544204
329415.6695190284 5213494.020080343 4272.932418249067 2523276.06302217 26064.078348305236 3527071.8815421415


In [26]:
# cost-benefit analysis function for se

def cost_benefit_analysis_se():
    scale = 1
#     sum_county = sum_county.replace(np.nan, 0)

    class cost_bef:
        def __init__(self, farm_op_cc, farm_op_nt, lt_save_cc, lt_save_nt, corn_pr, soy_pr, co2_pr, ind_EF, re_rate):

            # GHG in CO2 eq, county total: sum_soc, sum_leach, sum_n2o, sum_yld in unit 10^3t
            self.rot_ratio = sum_county['rot_ratio']
            self.con_ratio = 1 - sum_county['rot_ratio']
            self.cc_area = sum_county['corn_cc_area']
            self.nt_area = sum_county['corn_nt_area']
            self.cc_GHG_soc = sum_county_se['sum_cc_soc'] * 44 / 12
            self.cc_GHG_n2o = sum_county_se['sum_cc_n2o'] / 28 * 44 * 0.273
            self.cc_GHG_lch = sum_county_se['sum_cc_lch'] * ind_EF / 28 * 44 * 0.273
            self.cc_GHG = self.cc_GHG_soc + self.cc_GHG_n2o + self.cc_GHG_lch
            self.nt_GHG_soc = sum_county_se['sum_nt_soc'] * 44 / 12
            self.nt_GHG_n2o = sum_county_se['sum_nt_n2o'] / 28 * 44 * 0.273
            self.nt_GHG_lch = sum_county_se['sum_nt_lch'] * ind_EF / 28 * 44 * 0.273
            self.nt_GHG = self.nt_GHG_soc + self.nt_GHG_n2o + self.nt_GHG_lch
            # carbon benefits, in USD $
            self.cc_COB_soc = self.cc_GHG_soc * co2_pr
            self.cc_COB_n2o = self.cc_GHG_n2o * co2_pr + self.cc_GHG_lch * co2_pr
            self.cc_COB = self.cc_COB_soc + self.cc_COB_n2o
            self.nt_COB_soc = self.nt_GHG_soc * co2_pr
            self.nt_COB_n2o = self.nt_GHG_n2o * co2_pr + self.nt_GHG_lch * co2_pr
            self.nt_COB = self.nt_COB_soc + self.nt_COB_n2o

            # on-farm cost, in USD $
            self.cc_yld = corn_pr * sum_county_se['sum_yld_cc_con'] \
                           + corn_pr * sum_county_se['sum_yld_cc_rot_corn'] \
                           + soy_pr * sum_county_se['sum_yld_cc_rot_soy']
            self.cc_opt = farm_op_cc * sum_county['corn_cc_area']
            self.cc_sav = lt_save_cc * sum_county['corn_cc_area']
            self.cc_FC = self.cc_yld
            self.nt_yld = corn_pr * sum_county_se['sum_yld_nt_con'] \
                           + corn_pr * sum_county_se['sum_yld_nt_rot_corn'] \
                           + soy_pr * sum_county_se['sum_yld_nt_rot_soy']
            self.nt_opt = farm_op_nt * sum_county['corn_nt_area']
            self.nt_sav = lt_save_nt * sum_county['corn_nt_area']
            self.nt_FC = self.nt_yld

            # Social benefit, carbon benefits + yield return
            self.cc_SB = self.cc_COB * re_rate + self.cc_yld
            self.nt_SB = self.nt_COB * re_rate + self.nt_yld
            # Total return to farmer
            self.cc_MRG = self.cc_COB * re_rate + self.cc_FC
            self.nt_MRG = self.nt_COB * re_rate + self.nt_FC

            # CO2 mitigation intensity and efficiency at $/ha
            self.cc_intensity = self.cc_GHG / self.cc_area
            self.nt_intensity = self.nt_GHG / self.nt_area
            self.cc_cost = self.cc_FC / self.cc_area
            self.nt_cost = self.nt_FC / self.nt_area
            self.cc_efficiency = self.cc_FC / self.cc_GHG
            self.nt_efficiency = self.nt_FC / self.nt_GHG

            self.cc_intensity_us = self.cc_GHG.sum()  / sum_county['corn_cc_area'].sum()
            self.nt_intensity_us = self.nt_GHG.sum()  / sum_county['corn_nt_area'].sum()
            self.cc_efficiency_us = (self.cc_FC).sum() / (self.cc_GHG).sum()
            self.nt_efficiency_us = (self.nt_FC).sum() / (self.nt_GHG).sum()
            print(self.cc_intensity_us, self.nt_intensity_us, self.cc_efficiency_us, self.nt_efficiency_us)


    # set prices for farm_op_cc, farm_op_nt, lt_save_cc, lt_save_nt, corn_pr, soy_pr, co2_pr, ind_EF, re_rate
    margin = cost_bef(-98, 57, 32, -29, 143, 359, 46, 0.011, 0.6) # mean margin

    est_cty_cc = pd.concat((margin.cc_COB_soc, margin.cc_COB_n2o, margin.cc_COB, margin.cc_yld,
                            margin.cc_opt, margin.cc_sav, margin.cc_FC, margin.cc_SB, margin.cc_MRG), axis = 1)
    est_cty_nt = pd.concat((margin.nt_COB_soc, margin.nt_COB_n2o, margin.nt_COB, margin.nt_yld,
                            margin.nt_opt, margin.nt_sav, margin.nt_FC, margin.nt_SB, margin.nt_MRG), axis = 1)
    est_all = pd.concat((est_cty_cc / scale, est_cty_nt / scale, margin.cc_intensity, margin.nt_intensity, 
                         margin.cc_cost, margin.nt_cost, margin.cc_efficiency, margin.nt_efficiency), axis = 1)
    
    
    items = ['CB_SOC', 'CB_N2O', 'CB', 'YldReturn', 'Operation', 'Longterm', 'FC', 'SB', 'MRG']
    col_names = [head + item for head in ['CC_','NT_'] for item in items] + ['cc_int', 'nt_int', 'cc_cost', 'nt_cost', 'cc_eff', 'nt_eff']
    est_all.columns = col_names
    est_all['FIPS'] = sum_county_se['FIPS']
    est_all.to_csv(county_outputs_dir + 'cost_bef_historic_2017_se.csv')

0.6078615544160922 0.5555697649450924 16.218811602569055 19.711720684172292


In [27]:
# cost-benefit in optimized CC scenario

def optimize_CC(area, title):

    def cost_bef_opt_cc(adopt_rate_cc, adopt_rate_nt, farm_op_cc, farm_op_nt, lt_save_cc, lt_save_nt, corn_pr, soy_pr, co2_pr, ind_EF, re_rate):

        cc_area1 = area * sum_county['cc_ratio'].fillna(0)
        nt_area1 = area * sum_county['nt_ratio'].fillna(0)
        cc_area2 = area * adopt_rate_cc
        nt_area2 = area * adopt_rate_nt
        cc_area = pd.concat([cc_area1, cc_area2], axis = 1).max(axis = 1)
        nt_area = pd.concat([nt_area1, nt_area2], axis = 1).max(axis = 1)
        rot_ratio = pred_county['rotation_ratio'].fillna(0)
        con_ratio = (1-pred_county['rotation_ratio']).fillna(0)
        
        area_rot_cc = area * adopt_rate_cc * rot_ratio
        area_rot_nt = area * adopt_rate_nt * rot_ratio
        area_con_cc = area * adopt_rate_cc * (1-rot_ratio)
        area_con_nt = area * adopt_rate_nt * (1-rot_ratio)

        # aggreagate delta_SOC in corn-soy and corn-corn systems
        sum_soc_cc_rot = pred_county['Rot_delta_SOC_30'] * area_rot_cc
        sum_soc_nt_rot = pred_county['Rot_delta_SOC_60'] * area_rot_nt
        sum_soc_cc_con = pred_county['Rot_delta_SOC_30'] * area_con_cc
        sum_soc_nt_con = pred_county['Rot_delta_SOC_60'] * area_con_nt

        sum_cc_soc = sum_soc_cc_rot + sum_soc_cc_con
        sum_nt_soc = sum_soc_nt_rot + sum_soc_nt_con
        # aggreagate delta_N2O
        sum_n2o_cc_rot_corn = pred_county['Rot_delta_N2O_cc_corn'] * area_rot_cc / 2
        sum_n2o_cc_rot_soy = pred_county['Rot_delta_N2O_cc_soy'] * area_rot_cc / 2
        sum_n2o_nt_rot_corn = pred_county['Rot_delta_N2O_nt_corn'] * area_rot_nt / 2
        sum_n2o_nt_rot_soy = pred_county['Rot_delta_N2O_nt_soy'] * area_rot_nt / 2
        sum_n2o_cc_con = pred_county['Con_delta_N2O_cc'] * area_con_cc
        sum_n2o_nt_con = pred_county['Con_delta_N2O_nt'] * area_con_nt

        sum_n2o_cc_rot = sum_n2o_cc_rot_corn + sum_n2o_cc_rot_soy
        sum_n2o_nt_rot = sum_n2o_nt_rot_corn + sum_n2o_nt_rot_soy
        sum_cc_n2o = sum_n2o_cc_rot + sum_n2o_cc_con
        sum_nt_n2o = sum_n2o_nt_rot + sum_n2o_nt_con
        # aggreagate delta_N leaching
        sum_lch_cc_rot_corn = pred_county['Rot_delta_lch_cc_corn'] * area_rot_cc / 2
        sum_lch_cc_rot_soy = pred_county['Rot_delta_lch_cc_soy'] * area_rot_cc / 2
        sum_lch_nt_rot_corn = pred_county['Rot_delta_lch_nt_corn'] * area_rot_nt / 2
        sum_lch_nt_rot_soy = pred_county['Rot_delta_lch_nt_soy'] * area_rot_nt / 2
        sum_lch_cc_con = pred_county['Con_delta_lch_cc'] * area_con_cc
        sum_lch_nt_con = pred_county['Con_delta_lch_nt'] * area_con_nt
        
        sum_lch_cc_rot = sum_lch_cc_rot_corn + sum_lch_cc_rot_soy
        sum_lch_nt_rot = sum_lch_nt_rot_corn + sum_lch_nt_rot_soy
        sum_cc_lch = sum_lch_cc_rot + sum_lch_cc_con
        sum_nt_lch = sum_lch_nt_rot + sum_lch_nt_con
        # aggreagate delta_yield
        sum_yld_cc_rot_corn = pred_county['Rot_delta_yld_cc_corn'] * area_rot_cc / 2
        sum_yld_cc_rot_soy = pred_county['Rot_delta_yld_cc_soy'] * area_rot_cc / 2
        sum_yld_nt_rot_corn = pred_county['Rot_delta_yld_nt_corn'] * area_rot_nt / 2
        sum_yld_nt_rot_soy = pred_county['Rot_delta_yld_nt_soy'] * area_rot_nt / 2
        sum_yld_cc_con = pred_county['Con_delta_yld_cc'] * area_con_cc
        sum_yld_nt_con = pred_county['Con_delta_yld_nt'] * area_con_nt
        
        sum_yld_cc_rot = sum_yld_cc_rot_corn + sum_yld_cc_rot_soy
        sum_yld_nt_rot = sum_yld_nt_rot_corn + sum_yld_nt_rot_soy
        sum_cc_yld = sum_yld_cc_rot + sum_yld_cc_con
        sum_nt_yld = sum_yld_nt_rot + sum_yld_nt_con

#         sum_county_opt = pd.concat((sum_soc_cc_rot,sum_soc_nt_rot,sum_soc_cc_con,sum_soc_nt_con,
#             sum_n2o_cc_rot_corn,sum_n2o_cc_rot_soy,sum_n2o_nt_rot_corn,sum_n2o_nt_rot_soy,sum_n2o_cc_con,sum_n2o_nt_con,
#             sum_lch_cc_rot_corn,sum_lch_cc_rot_soy,sum_lch_nt_rot_corn,sum_lch_nt_rot_soy,sum_lch_cc_con,sum_lch_nt_con
#             sum_yld_cc_rot_corn,sum_yld_cc_rot_soy,sum_yld_nt_rot_corn,sum_yld_nt_rot_soy,sum_yld_cc_con,sum_yld_nt_con),
#                               axis = 1)
#         sum_county_opt.columns = ['sum_soc_cc_rot', 'sum_cc_soc_nt_rot', 'sum_soc_cc_con', 'sum_soc_nt_con',
#     'sum_n2o_cc_rot_corn','sum_n2o_cc_rot_soy','sum_n2o_nt_rot_corn','sum_n2o_nt_rot_soy','sum_n2o_cc_con','sum_n2o_nt_con',
#     'sum_lch_cc_rot_corn','sum_lch_cc_rot_soy','sum_lch_nt_rot_corn','sum_lch_nt_rot_soy','sum_lch_cc_con','sum_lch_nt_con',
#     'sum_yld_cc_rot_corn','sum_yld_cc_rot_soy','sum_yld_nt_rot_corn','sum_yld_nt_rot_soy','sum_yld_cc_con','sum_yld_nt_rot'], index=range(len(pred_nt_yld)))
#         sum_county['FIPS'] = pred_county['FIPS']
#         sum_county.to_csv(county_outputs_dir + 'county_aggregates_optimize_2017.csv')
        
        
        # GHG in CO2 eq, county total: sum_soc, sum_leach, sum_n2o, sum_yld in unit 10^3t
        cc_GHG_soc = sum_cc_soc * 44 / 12
        cc_GHG_n2o = - sum_cc_n2o / 28 * 44 * 0.273
        cc_GHG_lch = - sum_cc_lch * ind_EF / 28 * 44 * 0.273
        cc_GHG = cc_GHG_soc + cc_GHG_n2o + cc_GHG_lch
        nt_GHG_soc = sum_nt_soc * 44 / 12
        nt_GHG_n2o = - sum_nt_n2o / 28 * 44 * 0.273
        nt_GHG_lch = - sum_nt_lch * ind_EF / 28 * 44 * 0.273
        nt_GHG = nt_GHG_soc + nt_GHG_n2o + nt_GHG_lch
        # carbon benefits, in USD $
        cc_COB_soc = cc_GHG_soc * co2_pr
        cc_COB_n2o = cc_GHG_n2o * co2_pr + cc_GHG_lch * co2_pr
        cc_COB = cc_COB_soc + cc_COB_n2o
        nt_COB_soc = nt_GHG_soc * co2_pr
        nt_COB_n2o = nt_GHG_n2o * co2_pr + nt_GHG_lch * co2_pr
        nt_COB = nt_COB_soc + nt_COB_n2o

        # on-farm cost, in USD $
        cc_yld = (corn_pr * sum_yld_cc_con + corn_pr * sum_yld_cc_rot_corn \
                       + soy_pr * sum_yld_cc_rot_soy)
        cc_opt = farm_op_cc * cc_area
        cc_sav = lt_save_cc * cc_area
        cc_FC = cc_yld + cc_opt + cc_sav
        nt_yld = (corn_pr * sum_yld_nt_con + corn_pr * sum_yld_nt_rot_corn \
                       + soy_pr * sum_yld_nt_rot_soy)
        nt_opt = farm_op_nt * nt_area
        nt_sav = lt_save_nt * nt_area
        nt_FC = nt_yld + nt_opt + nt_sav

        # Social benefit, carbon benefits + yield return
        cc_SB = cc_COB * re_rate + cc_yld
        nt_SB = nt_COB * re_rate + nt_yld
        # Total return to farmer
        cc_MRG = cc_COB * re_rate + cc_FC
        nt_MRG = nt_COB * re_rate + nt_FC


        cc_intensity = cc_GHG / cc_area
        cc_efficiency = cc_FC / cc_GHG
        nt_intensity = nt_GHG / nt_area
        nt_efficiency = nt_FC / nt_GHG
        cc_cty_sum = pd.concat((cc_GHG, cc_FC), axis=1)
        nt_cty_sum = pd.concat((nt_GHG, nt_FC), axis=1)
        cc_nt_sum = (cc_cty_sum + nt_cty_sum).sum(skipna=True) * 0.5

        return cc_intensity, cc_efficiency, cc_cty_sum
    
    cur_cc_adopt = sum_county['cc_ratio']
    cur_nt_adopt = sum_county['nt_ratio']
    FIPS = sum_county['FIPS']
    data_comb = pd.DataFrame()
    for (adopt_rate_cc, adopt_rate_nt) in zip((cur_cc_adopt, 0.30, 0.80), (cur_nt_adopt, 0.30, 0.80)):
        cc_int, cc_eff, cty_sum_cc = cost_bef_opt_cc(adopt_rate_cc, adopt_rate_nt, -98, 57, 32, -6, 143, 359, 46, 0.011, 0.6)  # mean margin
        cc_int_mx, cc_eff_mx, cty_sum_cc_max = cost_bef_opt_cc(adopt_rate_cc, adopt_rate_nt, -40, 95, 42, 0, 136, 314, 167, 0.011, 0.6) # max margin
        cc_int_mn, cc_eff_mn, cty_sum_cc_min = cost_bef_opt_cc(adopt_rate_cc, adopt_rate_nt, -206, 36, 26, -12, 170, 468, 27, 0.011, 0.6) # min margin
        data_comb = pd.concat([data_comb, cty_sum_cc, cty_sum_cc_max, cty_sum_cc_min], axis = 1)
    data_priority = pd.concat([FIPS, cc_int, cc_eff, data_comb], axis = 1, ignore_index=True)

    data_priority.columns = ['FIPS', 'co2_int_cc', 'co2_eff_cc'] + \
            [prc + sce + var for prc in ['cr', '30', '50'] for sce in ['_bs', '_mx', '_mn'] for var in ['_ghg', '_fc']]
#     data_priority.to_csv(county_outputs_dir + 'CC_priority_inputs.csv')

    sn_int = pd.DataFrame(data_priority[['30_bs_ghg', '30_bs_fc', '50_bs_ghg', '50_bs_fc']].to_numpy() - data_priority[['cr_bs_ghg', 'cr_bs_fc', 'cr_bs_ghg', 'cr_bs_fc']].to_numpy())
    sn_int_mx = pd.DataFrame(data_priority[['30_mx_ghg', '30_mx_fc', '50_mx_ghg', '50_mx_fc']].to_numpy() - data_priority[['cr_mx_ghg', 'cr_mx_fc', 'cr_mx_ghg', 'cr_mx_fc']].to_numpy())
    sn_int_mn = pd.DataFrame(data_priority[['30_mn_ghg', '30_mn_fc', '50_mn_ghg', '50_mn_fc']].to_numpy() - data_priority[['cr_mn_ghg', 'cr_mn_fc', 'cr_mn_ghg', 'cr_mn_fc']].to_numpy())
    sn_eff = pd.DataFrame(data_priority[['30_bs_ghg', '30_bs_fc', '50_bs_ghg', '50_bs_fc']].to_numpy() - data_priority[['cr_bs_ghg', 'cr_bs_fc', 'cr_bs_ghg', 'cr_bs_fc']].to_numpy())
    sn_eff_mx = pd.DataFrame(data_priority[['30_mx_ghg', '30_mx_fc', '50_mx_ghg', '50_mx_fc']].to_numpy() - data_priority[['cr_mx_ghg', 'cr_mx_fc', 'cr_mx_ghg', 'cr_mx_fc']].to_numpy())
    sn_eff_mn = pd.DataFrame(data_priority[['30_mn_ghg', '30_mn_fc', '50_mn_ghg', '50_mn_fc']].to_numpy() - data_priority[['cr_mn_ghg', 'cr_mn_fc', 'cr_mn_ghg', 'cr_mn_fc']].to_numpy())
    rank_int = pd.concat([data_priority[['FIPS', 'co2_int_cc']], sn_int, sn_int_mx, sn_int_mn], axis = 1).sort_values(by=['co2_int_cc'], ascending=False).reset_index(drop=True)
    rank_eff_pos = pd.concat([data_priority[['FIPS', 'co2_int_cc', 'co2_eff_cc']], sn_eff, sn_eff_mx, sn_eff_mn], axis = 1).sort_values(by=['co2_eff_cc'], ascending=False).reset_index(drop=True)
    rank_eff_pos = rank_eff_pos.loc[rank_eff_pos['co2_int_cc'] > 0, :]
    rank_eff_neg = pd.concat([data_priority[['FIPS', 'co2_int_cc', 'co2_eff_cc']], sn_eff, sn_eff_mx, sn_eff_mn], axis = 1).sort_values(by=['co2_eff_cc']).reset_index(drop=True)
    rank_eff_neg = rank_eff_neg.loc[rank_eff_neg['co2_int_cc'] <= 0, :]
    rank_eff = pd.concat([rank_eff_pos.drop(['co2_int_cc'], axis = 1), rank_eff_neg.drop(['co2_int_cc'], axis = 1)]).reset_index(drop=True)
    rank_int.columns = ['FIPS_int', 'co2_int_cc'] + [col + price for price in ['', '_mx', '_mn'] for col in ['int_30_ghg', 'int_30_fc', 'int_50_ghg', 'int_50_fc']]
    rank_eff.columns = ['FIPS_eff', 'co2_eff_cc'] + [col + price for price in ['', '_mx', '_mn'] for col in ['eff_30_ghg', 'eff_30_fc', 'eff_50_ghg', 'eff_50_fc']]
    comb_sn = pd.concat([rank_int, rank_eff], axis = 1)

    comb_sn['int_30_fc'][comb_sn['int_30_ghg'] < 0] = 0
    comb_sn['int_30_ghg'][comb_sn['int_30_ghg'] < 0] = 0
    comb_sn['int_50_fc'][comb_sn['int_50_ghg'] < 0] = 0
    comb_sn['int_50_ghg'][comb_sn['int_50_ghg'] < 0] = 0
    comb_sn['int_30_fc_mx'][comb_sn['int_30_ghg_mx'] < 0] = 0
    comb_sn['int_30_ghg_mx'][comb_sn['int_30_ghg_mx'] < 0] = 0
    comb_sn['int_50_fc_mx'][comb_sn['int_50_ghg_mx'] < 0] = 0
    comb_sn['int_50_ghg_mx'][comb_sn['int_50_ghg_mx'] < 0] = 0
    comb_sn['int_30_fc_mn'][comb_sn['int_30_ghg_mn'] < 0] = 0
    comb_sn['int_30_ghg_mn'][comb_sn['int_30_ghg_mn'] < 0] = 0
    comb_sn['int_50_fc_mn'][comb_sn['int_50_ghg_mn'] < 0] = 0
    comb_sn['int_50_ghg_mn'][comb_sn['int_50_ghg_mn'] < 0] = 0

    comb_sn['eff_30_fc'][comb_sn['eff_30_ghg'] < 0] = 0
    comb_sn['eff_30_ghg'][comb_sn['eff_30_ghg'] < 0] = 0
    comb_sn['eff_50_fc'][comb_sn['eff_50_ghg'] < 0] = 0
    comb_sn['eff_50_ghg'][comb_sn['eff_50_ghg'] < 0] = 0
    comb_sn['eff_30_fc_mx'][comb_sn['eff_30_ghg_mx'] < 0] = 0
    comb_sn['eff_30_ghg_mx'][comb_sn['eff_30_ghg_mx'] < 0] = 0
    comb_sn['eff_50_fc_mx'][comb_sn['eff_50_ghg_mx'] < 0] = 0
    comb_sn['eff_50_ghg_mx'][comb_sn['eff_50_ghg_mx'] < 0] = 0
    comb_sn['eff_30_fc_mn'][comb_sn['eff_30_ghg_mn'] < 0] = 0
    comb_sn['eff_30_ghg_mn'][comb_sn['eff_30_ghg_mn'] < 0] = 0
    comb_sn['eff_50_fc_mn'][comb_sn['eff_50_ghg_mn'] < 0] = 0
    comb_sn['eff_50_ghg_mn'][comb_sn['eff_50_ghg_mn'] < 0] = 0

    comb_sn_cum = comb_sn.cumsum()
    comb_sn_cum.columns = [col + '_cum' for col in comb_sn.columns]
    priority_output = pd.concat([comb_sn, comb_sn_cum], axis = 1)
    priority_output.to_csv(county_outputs_dir + 'CC_priority_outputs_{}.csv'.format(title))


In [28]:
# cost-benefit in optimized NT scenario
def optimize_NT(area, title):

    def cost_bef_opt_nt(adopt_rate_cc, adopt_rate_nt, farm_op_cc, farm_op_nt, lt_save_cc, lt_save_nt, corn_pr, soy_pr, co2_pr, ind_EF, re_rate):

        cc_area1 = area * sum_county['cc_ratio'].fillna(0)
        nt_area1 = area * sum_county['nt_ratio'].fillna(0)
        cc_area2 = area * adopt_rate_cc
        nt_area2 = area * adopt_rate_nt
        cc_area = pd.concat([cc_area1, cc_area2], axis = 1).max(axis = 1)
        nt_area = pd.concat([nt_area1, nt_area2], axis = 1).max(axis = 1)
        rot_ratio = pred_county['rotation_ratio'].fillna(0)
        con_ratio = (1-pred_county['rotation_ratio']).fillna(0)
        
        area_rot_cc = area * adopt_rate_cc * rot_ratio
        area_rot_nt = area * adopt_rate_nt * rot_ratio
        area_con_cc = area * adopt_rate_cc * (1-rot_ratio)
        area_con_nt = area * adopt_rate_nt * (1-rot_ratio)

        # aggreagate delta_SOC in corn-soy and corn-corn systems
        sum_soc_cc_rot = pred_county['Rot_delta_SOC_30'] * area_rot_cc
        sum_soc_nt_rot = pred_county['Rot_delta_SOC_60'] * area_rot_nt
        sum_soc_cc_con = pred_county['Rot_delta_SOC_30'] * area_con_cc
        sum_soc_nt_con = pred_county['Rot_delta_SOC_60'] * area_con_nt

        sum_cc_soc = sum_soc_cc_rot + sum_soc_cc_con
        sum_nt_soc = sum_soc_nt_rot + sum_soc_nt_con
        # aggreagate delta_N2O
        sum_n2o_cc_rot_corn = pred_county['Rot_delta_N2O_cc_corn'] * area_rot_cc / 2
        sum_n2o_cc_rot_soy = pred_county['Rot_delta_N2O_cc_soy'] * area_rot_cc / 2
        sum_n2o_nt_rot_corn = pred_county['Rot_delta_N2O_nt_corn'] * area_rot_nt / 2
        sum_n2o_nt_rot_soy = pred_county['Rot_delta_N2O_nt_soy'] * area_rot_nt / 2
        sum_n2o_cc_con = pred_county['Con_delta_N2O_cc'] * area_con_cc
        sum_n2o_nt_con = pred_county['Con_delta_N2O_nt'] * area_con_nt

        sum_n2o_cc_rot = sum_n2o_cc_rot_corn + sum_n2o_cc_rot_soy
        sum_n2o_nt_rot = sum_n2o_nt_rot_corn + sum_n2o_nt_rot_soy
        sum_cc_n2o = sum_n2o_cc_rot + sum_n2o_cc_con
        sum_nt_n2o = sum_n2o_nt_rot + sum_n2o_nt_con
        # aggreagate delta_N leaching
        sum_lch_cc_rot_corn = pred_county['Rot_delta_lch_cc_corn'] * area_rot_cc / 2
        sum_lch_cc_rot_soy = pred_county['Rot_delta_lch_cc_soy'] * area_rot_cc / 2
        sum_lch_nt_rot_corn = pred_county['Rot_delta_lch_nt_corn'] * area_rot_nt / 2
        sum_lch_nt_rot_soy = pred_county['Rot_delta_lch_nt_soy'] * area_rot_nt / 2
        sum_lch_cc_con = pred_county['Con_delta_lch_cc'] * area_con_cc
        sum_lch_nt_con = pred_county['Con_delta_lch_nt'] * area_con_nt
        
        sum_lch_cc_rot = sum_lch_cc_rot_corn + sum_lch_cc_rot_soy
        sum_lch_nt_rot = sum_lch_nt_rot_corn + sum_lch_nt_rot_soy
        sum_cc_lch = sum_lch_cc_rot + sum_lch_cc_con
        sum_nt_lch = sum_lch_nt_rot + sum_lch_nt_con
        # aggreagate delta_yield
        sum_yld_cc_rot_corn = pred_county['Rot_delta_yld_cc_corn'] * area_rot_cc / 2
        sum_yld_cc_rot_soy = pred_county['Rot_delta_yld_cc_soy'] * area_rot_cc / 2
        sum_yld_nt_rot_corn = pred_county['Rot_delta_yld_nt_corn'] * area_rot_nt / 2
        sum_yld_nt_rot_soy = pred_county['Rot_delta_yld_nt_soy'] * area_rot_nt / 2
        sum_yld_cc_con = pred_county['Con_delta_yld_cc'] * area_con_cc
        sum_yld_nt_con = pred_county['Con_delta_yld_nt'] * area_con_nt
        
        sum_yld_cc_rot = sum_yld_cc_rot_corn + sum_yld_cc_rot_soy
        sum_yld_nt_rot = sum_yld_nt_rot_corn + sum_yld_nt_rot_soy
        sum_cc_yld = sum_yld_cc_rot + sum_yld_cc_con
        sum_nt_yld = sum_yld_nt_rot + sum_yld_nt_con
        
        
        # GHG in CO2 eq, county total: sum_soc, sum_leach, sum_n2o, sum_yld in unit 10^3t
        cc_GHG_soc = sum_cc_soc * 44 / 12
        cc_GHG_n2o = - sum_cc_n2o / 28 * 44 * 0.273
        cc_GHG_lch = - sum_cc_lch * ind_EF / 28 * 44 * 0.273
        cc_GHG = cc_GHG_soc + cc_GHG_n2o + cc_GHG_lch
        nt_GHG_soc = sum_nt_soc * 44 / 12
        nt_GHG_n2o = - sum_nt_n2o / 28 * 44 * 0.273
        nt_GHG_lch = - sum_nt_lch * ind_EF / 28 * 44 * 0.273
        nt_GHG = nt_GHG_soc + nt_GHG_n2o + nt_GHG_lch
        # carbon benefits, in USD $
        cc_COB_soc = cc_GHG_soc * co2_pr
        cc_COB_n2o = cc_GHG_n2o * co2_pr + cc_GHG_lch * co2_pr
        cc_COB = cc_COB_soc + cc_COB_n2o
        nt_COB_soc = nt_GHG_soc * co2_pr
        nt_COB_n2o = nt_GHG_n2o * co2_pr + nt_GHG_lch * co2_pr
        nt_COB = nt_COB_soc + nt_COB_n2o

        # on-farm cost, in USD $
        cc_yld = (corn_pr * sum_yld_cc_con + corn_pr * sum_yld_cc_rot_corn \
                       + soy_pr * sum_yld_cc_rot_soy)
        cc_opt = farm_op_cc * cc_area
        cc_sav = lt_save_cc * cc_area
        cc_FC = cc_yld + cc_opt + cc_sav
        nt_yld = (corn_pr * sum_yld_nt_con + corn_pr * sum_yld_nt_rot_corn \
                       + soy_pr * sum_yld_nt_rot_soy)
        nt_opt = farm_op_nt * nt_area
        nt_sav = lt_save_nt * nt_area
        nt_FC = nt_yld + nt_opt + nt_sav

        # Social benefit, carbon benefits + yield return
        cc_SB = cc_COB * re_rate + cc_yld
        nt_SB = nt_COB * re_rate + nt_yld
        # Total return to farmer
        cc_MRG = cc_COB * re_rate + cc_FC
        nt_MRG = nt_COB * re_rate + nt_FC

        cc_intensity = cc_GHG / cc_area
        cc_efficiency = cc_FC / cc_GHG
        nt_intensity = nt_GHG / nt_area
        nt_efficiency = nt_FC / nt_GHG
        cc_cty_sum = pd.concat((cc_GHG, cc_FC), axis=1)
        nt_cty_sum = pd.concat((nt_GHG, nt_FC), axis=1)
        cc_nt_sum = (cc_cty_sum + nt_cty_sum).sum(skipna=True) * 0.5

        return nt_intensity, nt_efficiency, nt_cty_sum

    cur_cc_adopt = sum_county['cc_ratio'].fillna(0)
    cur_nt_adopt = sum_county['nt_ratio'].fillna(0)
    FIPS = sum_county['FIPS']
    data_comb = pd.DataFrame()
    for (adopt_rate_cc, adopt_rate_nt) in zip((cur_cc_adopt, 0.31, 0.52), (cur_nt_adopt, 0.31, 0.52)):
        nt_int, nt_eff, cty_sum_nt = cost_bef_opt_nt(adopt_rate_cc, adopt_rate_nt, -98, 57, 32, -6, 143, 359, 46, 0.011, 0.6)  # mean margin
        nt_int_mx, nt_eff_mx, cty_sum_nt_max = cost_bef_opt_nt(adopt_rate_cc, adopt_rate_nt, -40, 95, 42, 0, 136, 314, 167, 0.011, 0.6) # max margin
        nt_int_mn, nt_eff_mn, cty_sum_nt_min = cost_bef_opt_nt(adopt_rate_cc, adopt_rate_nt, -206, 36, 26, -12, 170, 468, 27, 0.011, 0.6) # min margin
        data_comb = pd.concat([data_comb, cty_sum_nt, cty_sum_nt_max, cty_sum_nt_min], axis = 1)
    data_priority = pd.concat([FIPS, nt_int, nt_eff, data_comb], axis = 1, ignore_index=True)

    data_priority.columns = ['FIPS', 'co2_int_nt', 'co2_eff_nt'] + \
            [prc + sce + var for prc in ['cr', '30', '50'] for sce in ['_bs', '_mx', '_mn'] for var in ['_ghg', '_fc']]
    data_priority.to_csv(county_outputs_dir + 'NT_priority_inputs.csv')

    sn_int = pd.DataFrame(data_priority[['30_bs_ghg', '30_bs_fc', '50_bs_ghg', '50_bs_fc']].to_numpy() - data_priority[['cr_bs_ghg', 'cr_bs_fc', 'cr_bs_ghg', 'cr_bs_fc']].to_numpy())
    sn_int_mx = pd.DataFrame(data_priority[['30_mx_ghg', '30_mx_fc', '50_mx_ghg', '50_mx_fc']].to_numpy() - data_priority[['cr_mx_ghg', 'cr_mx_fc', 'cr_mx_ghg', 'cr_mx_fc']].to_numpy())
    sn_int_mn = pd.DataFrame(data_priority[['30_mn_ghg', '30_mn_fc', '50_mn_ghg', '50_mn_fc']].to_numpy() - data_priority[['cr_mn_ghg', 'cr_mn_fc', 'cr_mn_ghg', 'cr_mn_fc']].to_numpy())
    sn_eff = pd.DataFrame(data_priority[['30_bs_ghg', '30_bs_fc', '50_bs_ghg', '50_bs_fc']].to_numpy() - data_priority[['cr_bs_ghg', 'cr_bs_fc', 'cr_bs_ghg', 'cr_bs_fc']].to_numpy())
    sn_eff_mx = pd.DataFrame(data_priority[['30_mx_ghg', '30_mx_fc', '50_mx_ghg', '50_mx_fc']].to_numpy() - data_priority[['cr_mx_ghg', 'cr_mx_fc', 'cr_mx_ghg', 'cr_mx_fc']].to_numpy())
    sn_eff_mn = pd.DataFrame(data_priority[['30_mn_ghg', '30_mn_fc', '50_mn_ghg', '50_mn_fc']].to_numpy() - data_priority[['cr_mn_ghg', 'cr_mn_fc', 'cr_mn_ghg', 'cr_mn_fc']].to_numpy())
    rank_int = pd.concat([data_priority[['FIPS', 'co2_int_nt']], sn_int, sn_int_mx, sn_int_mn], axis = 1).sort_values(by=['co2_int_nt'], ascending=False).reset_index(drop=True)
    rank_eff_pos = pd.concat([data_priority[['FIPS', 'co2_int_nt', 'co2_eff_nt']], sn_eff, sn_eff_mx, sn_eff_mn], axis = 1).sort_values(by=['co2_eff_nt'], ascending=False).reset_index(drop=True)
    rank_eff_pos = rank_eff_pos.loc[rank_eff_pos['co2_int_nt'] > 0, :]
    rank_eff_neg = pd.concat([data_priority[['FIPS', 'co2_int_nt', 'co2_eff_nt']], sn_eff, sn_eff_mx, sn_eff_mn], axis = 1).sort_values(by=['co2_eff_nt']).reset_index(drop=True)
    rank_eff_neg = rank_eff_neg.loc[rank_eff_neg['co2_int_nt'] <= 0, :]
    rank_eff = pd.concat([rank_eff_pos.drop(['co2_int_nt'], axis = 1), rank_eff_neg.drop(['co2_int_nt'], axis = 1)]).reset_index(drop=True)
    rank_int.columns = ['FIPS_int', 'co2_int_nt'] + [col + price for price in ['', '_mx', '_mn'] for col in ['int_30_ghg', 'int_30_fc', 'int_50_ghg', 'int_50_fc']]
    rank_eff.columns = ['FIPS_eff', 'co2_eff_nt'] + [col + price for price in ['', '_mx', '_mn'] for col in ['eff_30_ghg', 'eff_30_fc', 'eff_50_ghg', 'eff_50_fc']]
    comb_sn = pd.concat([rank_int, rank_eff], axis = 1)
    comb_sn['int_30_fc'][comb_sn['int_30_ghg'] < 0] = 0
    comb_sn['int_30_ghg'][comb_sn['int_30_ghg'] < 0] = 0
    comb_sn['int_50_fc'][comb_sn['int_50_ghg'] < 0] = 0
    comb_sn['int_50_ghg'][comb_sn['int_50_ghg'] < 0] = 0
    comb_sn['int_30_fc_mx'][comb_sn['int_30_ghg_mx'] < 0] = 0
    comb_sn['int_30_ghg_mx'][comb_sn['int_30_ghg_mx'] < 0] = 0
    comb_sn['int_50_fc_mx'][comb_sn['int_50_ghg_mx'] < 0] = 0
    comb_sn['int_50_ghg_mx'][comb_sn['int_50_ghg_mx'] < 0] = 0
    comb_sn['int_30_fc_mn'][comb_sn['int_30_ghg_mn'] < 0] = 0
    comb_sn['int_30_ghg_mn'][comb_sn['int_30_ghg_mn'] < 0] = 0
    comb_sn['int_50_fc_mn'][comb_sn['int_50_ghg_mn'] < 0] = 0
    comb_sn['int_50_ghg_mn'][comb_sn['int_50_ghg_mn'] < 0] = 0

    comb_sn['eff_30_fc'][comb_sn['eff_30_ghg'] < 0] = 0
    comb_sn['eff_30_ghg'][comb_sn['eff_30_ghg'] < 0] = 0
    comb_sn['eff_50_fc'][comb_sn['eff_50_ghg'] < 0] = 0
    comb_sn['eff_50_ghg'][comb_sn['eff_50_ghg'] < 0] = 0
    comb_sn['eff_30_fc_mx'][comb_sn['eff_30_ghg_mx'] < 0] = 0
    comb_sn['eff_30_ghg_mx'][comb_sn['eff_30_ghg_mx'] < 0] = 0
    comb_sn['eff_50_fc_mx'][comb_sn['eff_50_ghg_mx'] < 0] = 0
    comb_sn['eff_50_ghg_mx'][comb_sn['eff_50_ghg_mx'] < 0] = 0
    comb_sn['eff_30_fc_mn'][comb_sn['eff_30_ghg_mn'] < 0] = 0
    comb_sn['eff_30_ghg_mn'][comb_sn['eff_30_ghg_mn'] < 0] = 0
    comb_sn['eff_50_fc_mn'][comb_sn['eff_50_ghg_mn'] < 0] = 0
    comb_sn['eff_50_ghg_mn'][comb_sn['eff_50_ghg_mn'] < 0] = 0

    comb_sn_cum = comb_sn.cumsum()
    comb_sn_cum.columns = [col + '_cum' for col in comb_sn.columns]
    priority_output = pd.concat([comb_sn, comb_sn_cum], axis = 1)
    priority_output.to_csv(county_outputs_dir + 'NT_priority_outputs_{}.csv'.format(title))


In [29]:
# determine the better practice (CC or NT) at a certain adopt rate that regarding soc, yield, GHG, return.
def est_dom(area,adopt_rate,farm_op_cc,farm_op_nt,lt_save_cc,lt_save_nt,corn_pr,soy_pr,co2_pr,ind_EF,re_rate,title):

    cc_area = area * adopt_rate / 100
    nt_area = area * adopt_rate / 100
    rot_ratio = pred_county['rotation_ratio'].fillna(0)
    con_ratio = (1-pred_county['rotation_ratio']).fillna(0)

    area_rot_cc = cc_area * rot_ratio
    area_rot_nt = nt_area * rot_ratio
    area_con_cc = cc_area * (1-rot_ratio)
    area_con_nt = nt_area * (1-rot_ratio)

    # aggreagate delta_SOC in corn-soy and corn-corn systems
    sum_soc_cc_rot = pred_county['Rot_delta_SOC_30'] * area_rot_cc
    sum_soc_nt_rot = pred_county['Rot_delta_SOC_60'] * area_rot_nt
    sum_soc_cc_con = pred_county['Rot_delta_SOC_30'] * area_con_cc
    sum_soc_nt_con = pred_county['Rot_delta_SOC_60'] * area_con_nt

    sum_cc_soc = sum_soc_cc_rot + sum_soc_cc_con
    sum_nt_soc = sum_soc_nt_rot + sum_soc_nt_con
    # aggreagate delta_N2O
    sum_n2o_cc_rot_corn = pred_county['Rot_delta_N2O_cc_corn'] * area_rot_cc / 2
    sum_n2o_cc_rot_soy = pred_county['Rot_delta_N2O_cc_soy'] * area_rot_cc / 2
    sum_n2o_nt_rot_corn = pred_county['Rot_delta_N2O_nt_corn'] * area_rot_nt / 2
    sum_n2o_nt_rot_soy = pred_county['Rot_delta_N2O_nt_soy'] * area_rot_nt / 2
    sum_n2o_cc_con = pred_county['Con_delta_N2O_cc'] * area_con_cc
    sum_n2o_nt_con = pred_county['Con_delta_N2O_nt'] * area_con_nt

    sum_n2o_cc_rot = sum_n2o_cc_rot_corn + sum_n2o_cc_rot_soy
    sum_n2o_nt_rot = sum_n2o_nt_rot_corn + sum_n2o_nt_rot_soy
    sum_cc_n2o = sum_n2o_cc_rot + sum_n2o_cc_con
    sum_nt_n2o = sum_n2o_nt_rot + sum_n2o_nt_con
    # aggreagate delta_N leaching
    sum_lch_cc_rot_corn = pred_county['Rot_delta_lch_cc_corn'] * area_rot_cc / 2
    sum_lch_cc_rot_soy = pred_county['Rot_delta_lch_cc_soy'] * area_rot_cc / 2
    sum_lch_nt_rot_corn = pred_county['Rot_delta_lch_nt_corn'] * area_rot_nt / 2
    sum_lch_nt_rot_soy = pred_county['Rot_delta_lch_nt_soy'] * area_rot_nt / 2
    sum_lch_cc_con = pred_county['Con_delta_lch_cc'] * area_con_cc
    sum_lch_nt_con = pred_county['Con_delta_lch_nt'] * area_con_nt

    sum_lch_cc_rot = sum_lch_cc_rot_corn + sum_lch_cc_rot_soy
    sum_lch_nt_rot = sum_lch_nt_rot_corn + sum_lch_nt_rot_soy
    sum_cc_lch = sum_lch_cc_rot + sum_lch_cc_con
    sum_nt_lch = sum_lch_nt_rot + sum_lch_nt_con
    # aggreagate delta_yield
    sum_yld_cc_rot_corn = pred_county['Rot_delta_yld_cc_corn'] * area_rot_cc / 2
    sum_yld_cc_rot_soy = pred_county['Rot_delta_yld_cc_soy'] * area_rot_cc / 2
    sum_yld_nt_rot_corn = pred_county['Rot_delta_yld_nt_corn'] * area_rot_nt / 2
    sum_yld_nt_rot_soy = pred_county['Rot_delta_yld_nt_soy'] * area_rot_nt / 2
    sum_yld_cc_con = pred_county['Con_delta_yld_cc'] * area_con_cc
    sum_yld_nt_con = pred_county['Con_delta_yld_nt'] * area_con_nt

    sum_yld_cc_rot = sum_yld_cc_rot_corn + sum_yld_cc_rot_soy
    sum_yld_nt_rot = sum_yld_nt_rot_corn + sum_yld_nt_rot_soy
    sum_cc_yld = sum_yld_cc_rot + sum_yld_cc_con
    sum_nt_yld = sum_yld_nt_rot + sum_yld_nt_con


    # GHG in CO2 eq, county total: sum_soc, sum_leach, sum_n2o, sum_yld in unit 10^3t
    cc_GHG_soc = sum_cc_soc * 44 / 12
    cc_GHG_n2o = - sum_cc_n2o / 28 * 44 * 0.273
    cc_GHG_lch = - sum_cc_lch * ind_EF / 28 * 44 * 0.273
    cc_GHG = cc_GHG_soc + cc_GHG_n2o + cc_GHG_lch
    nt_GHG_soc = sum_nt_soc * 44 / 12
    nt_GHG_n2o = - sum_nt_n2o / 28 * 44 * 0.273
    nt_GHG_lch = - sum_nt_lch * ind_EF / 28 * 44 * 0.273
    nt_GHG = nt_GHG_soc + nt_GHG_n2o + nt_GHG_lch
    # carbon benefits, in USD $
    cc_COB_soc = cc_GHG_soc * co2_pr
    cc_COB_n2o = cc_GHG_n2o * co2_pr + cc_GHG_lch * co2_pr
    cc_COB = cc_COB_soc + cc_COB_n2o
    nt_COB_soc = nt_GHG_soc * co2_pr
    nt_COB_n2o = nt_GHG_n2o * co2_pr + nt_GHG_lch * co2_pr
    nt_COB = nt_COB_soc + nt_COB_n2o

    # on-farm cost, in USD $
    cc_yld = (corn_pr * sum_yld_cc_con + corn_pr * sum_yld_cc_rot_corn \
                   + soy_pr * sum_yld_cc_rot_soy)
    cc_opt = farm_op_cc * cc_area
    cc_sav = lt_save_cc * cc_area
    cc_FC = cc_yld + cc_opt + cc_sav
    nt_yld = (corn_pr * sum_yld_nt_con + corn_pr * sum_yld_nt_rot_corn \
                   + soy_pr * sum_yld_nt_rot_soy)
    nt_opt = farm_op_nt * nt_area
    nt_sav = lt_save_nt * nt_area
    nt_FC = nt_yld + nt_opt + nt_sav

    # Social benefit, carbon benefits + yield return
    cc_SB = cc_COB * re_rate + cc_yld
    nt_SB = nt_COB * re_rate + nt_yld
    # Total return to farmer
    cc_MRG = cc_COB * re_rate + cc_FC
    nt_MRG = nt_COB * re_rate + nt_FC

    cc_eff = cc_FC / cc_GHG
    nt_eff = nt_FC / nt_GHG
    cc_eff_us = cc_FC.sum() / cc_GHG.sum()
    nt_eff_us = nt_FC.sum() / nt_GHG.sum()
#     cc_cty_sum = pd.concat((cc_GHG, cc_FC), axis=1)
#     nt_cty_sum = pd.concat((nt_GHG, nt_FC), axis=1)
#     cc_nt_sum = (cc_cty_sum + nt_cty_sum).sum(skipna=True) * 0.5

    data_dom = pd.concat((sum_nt_soc.rename('pred_nt_soc'),sum_cc_soc.rename('pred_cc_soc'),
                          sum_nt_yld.rename('pred_nt_yld'),sum_cc_yld.rename('pred_cc_yld'),
                          nt_GHG.rename('pred_nt_ghg'),cc_GHG.rename('pred_cc_ghg'),
                          nt_eff.rename('pred_nt_eff'),cc_eff.rename('pred_cc_eff'),
                          nt_SB.rename('pred_nt_eco'),cc_SB.rename('pred_cc_eco'), 
                          nt_MRG.rename('pred_nt_mrg'),cc_MRG.rename('pred_cc_mrg')), axis = 1)
    data_dom['FIPS'] = pred_county['FIPS']
    data_dom = data_dom.fillna(0)
    soc_dom = (data_dom['pred_nt_soc'] > data_dom['pred_cc_soc']).astype(int)
    yld_dom = (data_dom['pred_nt_yld'] > data_dom['pred_cc_yld']).astype(int)
    ghg_dom = (data_dom['pred_nt_ghg'] > data_dom['pred_cc_ghg']).astype(int)
    eff_dom = (data_dom['pred_nt_eff'] > data_dom['pred_cc_eff']).astype(int)
    eco_dom = (data_dom['pred_nt_eco'] > data_dom['pred_cc_eco']).astype(int)
    mrg_dom = (data_dom['pred_nt_mrg'] > data_dom['pred_cc_mrg']).astype(int)

    est_dom = pd.concat((data_dom, soc_dom.rename('soc_dom'), yld_dom.rename('yld_dom'), ghg_dom.rename('ghg_dom'), eff_dom.rename('eff_dom'), eco_dom.rename('eco_dom'), mrg_dom.rename('mrg_dom')), axis = 1)
    # est_dom = est_dom[corn_area > 0]
    print(len(corn_area.dropna()), (est_dom['soc_dom'] == 1).sum(), (est_dom['yld_dom'] == 1).sum(),
          (est_dom['ghg_dom'] == 1).sum(), (est_dom['eff_dom'] == 1).sum(), (est_dom['eco_dom'] == 1).sum(), (est_dom['mrg_dom'] == 1).sum())
    est_dom.to_csv(county_outputs_dir + 'county_dom_practice_{}.csv'.format(title))


1481 1455 588 1481 1418 1071 1479
1481 3012 1201 3053 2950 2184 3043


In [30]:
# the response of GHG emissions to on-farm costs (investments) under spatially-prioritized scenario

def Total_mitigation(area, est_dom, adopt_rate_cc, adopt_rate_nt, farm_op_cc, farm_op_nt, lt_save_cc, lt_save_nt, \
                     corn_pr, soy_pr, co2_pr, ind_EF, re_rate):

    cc_area1 = area * sum_county['cc_ratio'].fillna(0)
    nt_area1 = area * sum_county['nt_ratio'].fillna(0)
    cc_area2 = area * adopt_rate_cc
    nt_area2 = area * adopt_rate_nt
    cc_area = pd.concat([cc_area1, cc_area2], axis = 1).max(axis = 1)
    nt_area = pd.concat([nt_area1, nt_area2], axis = 1).max(axis = 1)
    rot_ratio = pred_county['rotation_ratio'].fillna(0)
    con_ratio = (1-pred_county['rotation_ratio']).fillna(0)

    area_rot_cc = cc_area * rot_ratio
    area_rot_nt = nt_area * rot_ratio
    area_con_cc = cc_area * (1-rot_ratio)
    area_con_nt = nt_area * (1-rot_ratio)

    # aggreagate delta_SOC in corn-soy and corn-corn systems
    sum_soc_cc_rot = pred_county['Rot_delta_SOC_30'] * area_rot_cc
    sum_soc_nt_rot = pred_county['Rot_delta_SOC_60'] * area_rot_nt
    sum_soc_cc_con = pred_county['Rot_delta_SOC_30'] * area_con_cc
    sum_soc_nt_con = pred_county['Rot_delta_SOC_60'] * area_con_nt

    sum_cc_soc = sum_soc_cc_rot + sum_soc_cc_con
    sum_nt_soc = sum_soc_nt_rot + sum_soc_nt_con
    # aggreagate delta_N2O
    sum_n2o_cc_rot_corn = pred_county['Rot_delta_N2O_cc_corn'] * area_rot_cc / 2
    sum_n2o_cc_rot_soy = pred_county['Rot_delta_N2O_cc_soy'] * area_rot_cc / 2
    sum_n2o_nt_rot_corn = pred_county['Rot_delta_N2O_nt_corn'] * area_rot_nt / 2
    sum_n2o_nt_rot_soy = pred_county['Rot_delta_N2O_nt_soy'] * area_rot_nt / 2
    sum_n2o_cc_con = pred_county['Con_delta_N2O_cc'] * area_con_cc
    sum_n2o_nt_con = pred_county['Con_delta_N2O_nt'] * area_con_nt

    sum_n2o_cc_rot = sum_n2o_cc_rot_corn + sum_n2o_cc_rot_soy
    sum_n2o_nt_rot = sum_n2o_nt_rot_corn + sum_n2o_nt_rot_soy
    sum_cc_n2o = sum_n2o_cc_rot + sum_n2o_cc_con
    sum_nt_n2o = sum_n2o_nt_rot + sum_n2o_nt_con
    # aggreagate delta_N leaching
    sum_lch_cc_rot_corn = pred_county['Rot_delta_lch_cc_corn'] * area_rot_cc / 2
    sum_lch_cc_rot_soy = pred_county['Rot_delta_lch_cc_soy'] * area_rot_cc / 2
    sum_lch_nt_rot_corn = pred_county['Rot_delta_lch_nt_corn'] * area_rot_nt / 2
    sum_lch_nt_rot_soy = pred_county['Rot_delta_lch_nt_soy'] * area_rot_nt / 2
    sum_lch_cc_con = pred_county['Con_delta_lch_cc'] * area_con_cc
    sum_lch_nt_con = pred_county['Con_delta_lch_nt'] * area_con_nt

    sum_lch_cc_rot = sum_lch_cc_rot_corn + sum_lch_cc_rot_soy
    sum_lch_nt_rot = sum_lch_nt_rot_corn + sum_lch_nt_rot_soy
    sum_cc_lch = sum_lch_cc_rot + sum_lch_cc_con
    sum_nt_lch = sum_lch_nt_rot + sum_lch_nt_con
    # aggreagate delta_yield
    sum_yld_cc_rot_corn = pred_county['Rot_delta_yld_cc_corn'] * area_rot_cc / 2
    sum_yld_cc_rot_soy = pred_county['Rot_delta_yld_cc_soy'] * area_rot_cc / 2
    sum_yld_nt_rot_corn = pred_county['Rot_delta_yld_nt_corn'] * area_rot_nt / 2
    sum_yld_nt_rot_soy = pred_county['Rot_delta_yld_nt_soy'] * area_rot_nt / 2
    sum_yld_cc_con = pred_county['Con_delta_yld_cc'] * area_con_cc
    sum_yld_nt_con = pred_county['Con_delta_yld_nt'] * area_con_nt

    sum_yld_cc_rot = sum_yld_cc_rot_corn + sum_yld_cc_rot_soy
    sum_yld_nt_rot = sum_yld_nt_rot_corn + sum_yld_nt_rot_soy
    sum_cc_yld = sum_yld_cc_rot + sum_yld_cc_con
    sum_nt_yld = sum_yld_nt_rot + sum_yld_nt_con

    
    # GHG in CO2 eq, county total: sum_soc, sum_leach, sum_n2o, sum_yld in unit 10^3t
    cc_GHG_soc = sum_cc_soc * 44 / 12
    cc_GHG_n2o = - sum_cc_n2o / 28 * 44 * 0.273
    cc_GHG_lch = - sum_cc_lch * ind_EF / 28 * 44 * 0.273
    cc_GHG = cc_GHG_soc + cc_GHG_n2o + cc_GHG_lch
    nt_GHG_soc = sum_nt_soc * 44 / 12
    nt_GHG_n2o = - sum_nt_n2o / 28 * 44 * 0.273
    nt_GHG_lch = - sum_nt_lch * ind_EF / 28 * 44 * 0.273
    nt_GHG = nt_GHG_soc + nt_GHG_n2o + nt_GHG_lch
    # carbon benefits, in USD $
    cc_COB_soc = cc_GHG_soc * co2_pr
    cc_COB_n2o = cc_GHG_n2o * co2_pr + cc_GHG_lch * co2_pr
    cc_COB = cc_COB_soc + cc_COB_n2o
    nt_COB_soc = nt_GHG_soc * co2_pr
    nt_COB_n2o = nt_GHG_n2o * co2_pr + nt_GHG_lch * co2_pr
    nt_COB = nt_COB_soc + nt_COB_n2o

    # on-farm cost, in USD $
    cc_yld = (corn_pr * sum_yld_cc_con + corn_pr * sum_yld_cc_rot_corn \
                   + soy_pr * sum_yld_cc_rot_soy)
    cc_opt = farm_op_cc * cc_area
    cc_sav = lt_save_cc * cc_area
    cc_FC = cc_yld + cc_opt + cc_sav
    nt_yld = (corn_pr * sum_yld_nt_con + corn_pr * sum_yld_nt_rot_corn \
                   + soy_pr * sum_yld_nt_rot_soy)
    nt_opt = farm_op_nt * nt_area
    nt_sav = lt_save_nt * nt_area
    nt_FC = nt_yld + nt_opt + nt_sav

    # Social benefit, carbon benefits + yield return
    cc_SB = cc_COB * re_rate + cc_yld
    nt_SB = nt_COB * re_rate + nt_yld
    # Total return to farmer
    cc_MRG = cc_COB * re_rate + cc_FC
    nt_MRG = nt_COB * re_rate + nt_FC

    # CO2 mitigation efficiency /ha and cost
    cc_intensity = cc_GHG / cc_area
    cc_efficiency = cc_FC / cc_GHG
    nt_intensity = nt_GHG / nt_area
    nt_efficiency = nt_FC / nt_GHG

    cc_cty_sum = pd.concat((sum_cc_soc, sum_cc_yld, cc_GHG, cc_FC, cc_SB, cc_MRG), axis = 1)
    nt_cty_sum = pd.concat((sum_nt_soc, sum_nt_yld, nt_GHG, nt_FC, nt_SB, nt_MRG), axis = 1)
    cc_nt_sum = (cc_cty_sum + nt_cty_sum).sum(skipna=True) * 0.5
    # a = cc_cty_sum[est_dom['soc_dom'] == 0].fillna(0)
    # b = nt_cty_sum[est_dom['soc_dom'] == 1].fillna(0)
    # print(cc_MRG.sum(), nt_MRG.sum())
    # cty_sum = pd.concat([cc_cty_sum, nt_cty_sum], axis = 1)
    # columns = ['cc_soc', 'cc_yld', 'cc_ghg', 'cc_eff', 'cc_sb', 'cc_mrg', 'nt_soc', 'nt_yld', 'nt_ghg', 'nt_eff', 'nt_sb', 'nt_mrg']
    # cty_sum.columns = columns
    # cty_sum.to_csv(plot_dir + 'outputs_stat/cost_bef_base_current.csv')
    soc_dom_sum = cc_cty_sum[est_dom['soc_dom'] == 0].reindex(range(3108),fill_value=0).add(nt_cty_sum[est_dom['soc_dom'] == 1].reindex(range(3108),fill_value=0)).sum(skipna=True)
    yld_dom_sum = cc_cty_sum[est_dom['yld_dom'] == 0].reindex(range(3108),fill_value=0).add(nt_cty_sum[est_dom['yld_dom'] == 1].reindex(range(3108),fill_value=0)).sum(skipna=True)
    int_dom_sum = cc_cty_sum[est_dom['ghg_dom'] == 0].reindex(range(3108),fill_value=0).add(nt_cty_sum[est_dom['ghg_dom'] == 1].reindex(range(3108),fill_value=0)).sum(skipna=True)
    eff_dom_sum = cc_cty_sum[est_dom['eff_dom'] == 0].reindex(range(3108),fill_value=0).add(nt_cty_sum[est_dom['eff_dom'] == 1].reindex(range(3108),fill_value=0)).sum(skipna=True)
    sb_dom_sum = cc_cty_sum[est_dom['eco_dom'] == 0].reindex(range(3108),fill_value=0).add(nt_cty_sum[est_dom['eco_dom'] == 1].reindex(range(3108),fill_value=0)).sum(skipna=True)
    mrg_dom_sum = cc_cty_sum[est_dom['mrg_dom'] == 0].reindex(range(3108),fill_value=0).add(nt_cty_sum[est_dom['mrg_dom'] == 1].reindex(range(3108),fill_value=0)).sum(skipna=True)

    cc_cty_sum = cc_cty_sum.sum(skipna=True)
    nt_cty_sum = nt_cty_sum.sum(skipna=True)
    return (cc_cty_sum, nt_cty_sum, int_dom_sum, eff_dom_sum)

def Total_mitigation_se(area, est_dom, adopt_rate_cc, adopt_rate_nt, farm_op_cc, farm_op_nt, lt_save_cc, lt_save_nt,\
                        corn_pr, soy_pr, co2_pr, ind_EF, re_rate):

    cc_area1 = area * sum_county['cc_ratio'].fillna(0)
    nt_area1 = area * sum_county['nt_ratio'].fillna(0)
    cc_area2 = area * adopt_rate_cc
    nt_area2 = area * adopt_rate_nt
    cc_area = pd.concat([cc_area1, cc_area2], axis = 1).max(axis = 1)
    nt_area = pd.concat([nt_area1, nt_area2], axis = 1).max(axis = 1)
    rot_ratio = pred_county_se['rotation_ratio'].fillna(0)
    con_ratio = (1-pred_county_se['rotation_ratio']).fillna(0)

    area_rot_cc = cc_area * rot_ratio
    area_rot_nt = nt_area * rot_ratio
    area_con_cc = cc_area * (1-rot_ratio)
    area_con_nt = nt_area * (1-rot_ratio)

    # aggreagate delta_SOC in corn-soy and corn-corn systems
    sum_soc_cc_rot = pred_county_se['Rot_delta_SOC_30'] * area_rot_cc
    sum_soc_nt_rot = pred_county_se['Rot_delta_SOC_30'] * area_rot_nt
    sum_soc_cc_con = pred_county_se['Rot_delta_SOC_30'] * area_con_cc
    sum_soc_nt_con = pred_county_se['Rot_delta_SOC_30'] * area_con_nt

    sum_cc_soc = sum_soc_cc_rot + sum_soc_cc_con
    sum_nt_soc = sum_soc_nt_rot + sum_soc_nt_con
    # aggreagate delta_N2O
    sum_n2o_cc_rot_corn = pred_county_se['Rot_delta_N2O_cc_corn'] * area_rot_cc / 2
    sum_n2o_cc_rot_soy = pred_county_se['Rot_delta_N2O_cc_soy'] * area_rot_cc / 2
    sum_n2o_nt_rot_corn = pred_county_se['Rot_delta_N2O_nt_corn'] * area_rot_nt / 2
    sum_n2o_nt_rot_soy = pred_county_se['Rot_delta_N2O_nt_soy'] * area_rot_nt / 2
    sum_n2o_cc_con = pred_county_se['Con_delta_N2O_cc'] * area_con_cc
    sum_n2o_nt_con = pred_county_se['Con_delta_N2O_nt'] * area_con_nt

    sum_n2o_cc_rot = sum_n2o_cc_rot_corn + sum_n2o_cc_rot_soy
    sum_n2o_nt_rot = sum_n2o_nt_rot_corn + sum_n2o_nt_rot_soy
    sum_cc_n2o = sum_n2o_cc_rot + sum_n2o_cc_con
    sum_nt_n2o = sum_n2o_nt_rot + sum_n2o_nt_con
    # aggreagate delta_N leaching
    sum_lch_cc_rot_corn = pred_county_se['Rot_delta_lch_cc_corn'] * area_rot_cc / 2
    sum_lch_cc_rot_soy = pred_county_se['Rot_delta_lch_cc_soy'] * area_rot_cc / 2
    sum_lch_nt_rot_corn = pred_county_se['Rot_delta_lch_nt_corn'] * area_rot_nt / 2
    sum_lch_nt_rot_soy = pred_county_se['Rot_delta_lch_nt_soy'] * area_rot_nt / 2
    sum_lch_cc_con = pred_county_se['Con_delta_lch_cc'] * area_con_cc
    sum_lch_nt_con = pred_county_se['Con_delta_lch_nt'] * area_con_nt

    sum_lch_cc_rot = sum_lch_cc_rot_corn + sum_lch_cc_rot_soy
    sum_lch_nt_rot = sum_lch_nt_rot_corn + sum_lch_nt_rot_soy
    sum_cc_lch = sum_lch_cc_rot + sum_lch_cc_con
    sum_nt_lch = sum_lch_nt_rot + sum_lch_nt_con
    # aggreagate delta_yield
    sum_yld_cc_rot_corn = pred_county_se['Rot_delta_yld_cc_corn'] * area_rot_cc / 2
    sum_yld_cc_rot_soy = pred_county_se['Rot_delta_yld_cc_soy'] * area_rot_cc / 2
    sum_yld_nt_rot_corn = pred_county_se['Rot_delta_yld_nt_corn'] * area_rot_nt / 2
    sum_yld_nt_rot_soy = pred_county_se['Rot_delta_yld_nt_soy'] * area_rot_nt / 2
    sum_yld_cc_con = pred_county_se['Con_delta_yld_cc'] * area_con_cc
    sum_yld_nt_con = pred_county_se['Con_delta_yld_nt'] * area_con_nt

    sum_yld_cc_rot = sum_yld_cc_rot_corn + sum_yld_cc_rot_soy
    sum_yld_nt_rot = sum_yld_nt_rot_corn + sum_yld_nt_rot_soy
    sum_cc_yld = sum_yld_cc_rot + sum_yld_cc_con
    sum_nt_yld = sum_yld_nt_rot + sum_yld_nt_con


    # GHG in CO2 eq, county total: sum_soc, sum_leach, sum_n2o, sum_yld in unit 10^3t
    cc_GHG_soc = sum_cc_soc * 44 / 12
    cc_GHG_n2o = - sum_cc_n2o / 28 * 44 * 0.273
    cc_GHG_lch = - sum_cc_lch * ind_EF / 28 * 44 * 0.273
    cc_GHG = cc_GHG_soc + cc_GHG_n2o + cc_GHG_lch
    nt_GHG_soc = sum_nt_soc * 44 / 12
    nt_GHG_n2o = - sum_nt_n2o / 28 * 44 * 0.273
    nt_GHG_lch = - sum_nt_lch * ind_EF / 28 * 44 * 0.273
    nt_GHG = nt_GHG_soc + nt_GHG_n2o + nt_GHG_lch
    # carbon benefits, in USD $
    cc_COB_soc = cc_GHG_soc * co2_pr
    cc_COB_n2o = cc_GHG_n2o * co2_pr + cc_GHG_lch * co2_pr
    cc_COB = cc_COB_soc + cc_COB_n2o
    nt_COB_soc = nt_GHG_soc * co2_pr
    nt_COB_n2o = nt_GHG_n2o * co2_pr + nt_GHG_lch * co2_pr
    nt_COB = nt_COB_soc + nt_COB_n2o

    # on-farm cost, in USD $
    cc_yld = (corn_pr * sum_yld_cc_con + corn_pr * sum_yld_cc_rot_corn \
                   + soy_pr * sum_yld_cc_rot_soy)
    cc_opt = farm_op_cc * cc_area
    cc_sav = lt_save_cc * cc_area
    cc_FC = cc_yld + cc_opt + cc_sav
    nt_yld = (corn_pr * sum_yld_nt_con + corn_pr * sum_yld_nt_rot_corn \
                   + soy_pr * sum_yld_nt_rot_soy)
    nt_opt = farm_op_nt * nt_area
    nt_sav = lt_save_nt * nt_area
    nt_FC = nt_yld + nt_opt + nt_sav

    # Social benefit, carbon benefits + yield return
    cc_SB = cc_COB * re_rate + cc_yld
    nt_SB = nt_COB * re_rate + nt_yld
    # Total return to farmer
    cc_MRG = cc_COB * re_rate + cc_FC
    nt_MRG = nt_COB * re_rate + nt_FC

    # CO2 mitigation efficiency /ha and cost
    cc_intensity = cc_GHG / cc_area
    cc_efficiency = cc_FC / cc_GHG
    nt_intensity = nt_GHG / nt_area
    nt_efficiency = nt_FC / nt_GHG

    cc_cty_sum = pd.concat((sum_cc_soc, sum_cc_yld, cc_GHG, cc_FC, cc_SB, cc_MRG), axis = 1)
    nt_cty_sum = pd.concat((sum_nt_soc, sum_nt_yld, nt_GHG, nt_FC, nt_SB, nt_MRG), axis = 1)
    cc_nt_sum = (cc_cty_sum + nt_cty_sum).sum(skipna=True) * 0.5
    # a = cc_cty_sum[est_dom['soc_dom'] == 0].fillna(0)
    # b = nt_cty_sum[est_dom['soc_dom'] == 1].fillna(0)
    # print(cc_MRG.sum(), nt_MRG.sum())
    # cty_sum = pd.concat([cc_cty_sum, nt_cty_sum], axis = 1)
    # columns = ['cc_soc', 'cc_yld', 'cc_ghg', 'cc_eff', 'cc_sb', 'cc_mrg', 'nt_soc', 'nt_yld', 'nt_ghg', 'nt_eff', 'nt_sb', 'nt_mrg']
    # cty_sum.columns = columns
    # cty_sum.to_csv(plot_dir + 'outputs_stat/cost_bef_base_current.csv')
    soc_dom_sum = cc_cty_sum[est_dom['soc_dom'] == 0].reindex(range(3108),fill_value=0).add(nt_cty_sum[est_dom['soc_dom'] == 1].reindex(range(3108),fill_value=0)).sum(skipna=True)
    yld_dom_sum = cc_cty_sum[est_dom['yld_dom'] == 0].reindex(range(3108),fill_value=0).add(nt_cty_sum[est_dom['yld_dom'] == 1].reindex(range(3108),fill_value=0)).sum(skipna=True)
    int_dom_sum = cc_cty_sum[est_dom['ghg_dom'] == 0].reindex(range(3108),fill_value=0).add(nt_cty_sum[est_dom['ghg_dom'] == 1].reindex(range(3108),fill_value=0)).sum(skipna=True)
    eff_dom_sum = cc_cty_sum[est_dom['eff_dom'] == 0].reindex(range(3108),fill_value=0).add(nt_cty_sum[est_dom['eff_dom'] == 1].reindex(range(3108),fill_value=0)).sum(skipna=True)
    sb_dom_sum = cc_cty_sum[est_dom['eco_dom'] == 0].reindex(range(3108),fill_value=0).add(nt_cty_sum[est_dom['eco_dom'] == 1].reindex(range(3108),fill_value=0)).sum(skipna=True)
    mrg_dom_sum = cc_cty_sum[est_dom['mrg_dom'] == 0].reindex(range(3108),fill_value=0).add(nt_cty_sum[est_dom['mrg_dom'] == 1].reindex(range(3108),fill_value=0)).sum(skipna=True)

    cc_cty_sum = cc_cty_sum.sum(skipna=True)
    nt_cty_sum = nt_cty_sum.sum(skipna=True)
    return (cc_cty_sum, nt_cty_sum, int_dom_sum, eff_dom_sum)

In [ ]:
# conduct the defined cost benefit analysis
if __name__=="__main__": 
    sum_county = pd.read_csv(county_outputs_dir + 'county_aggregates_2017.csv')
    sum_county_se = pd.read_csv(county_outputs_dir + 'county_aggregates_2017_se.csv')
    cost_benefit_analysis()
    cost_benefit_analysis_se()

    corn_area = sum_county['corn_area']
    crop_area = sum_county['crop_area']
    optimize_CC(corn_area, 'cornfield')
    optimize_CC(crop_area, 'cropland')
    optimize_NT(corn_area, 'cornfield')
    optimize_NT(crop_area, 'cropland')

    est_dom(corn_area, 1, -98, 57, 32, -29, 143, 359, 46, 0.011, 0.6, 'cornfield')
    est_dom(crop_area, 1, -98, 57, 32, -29, 143, 359, 46, 0.011, 0.6, 'cropland')

In [ ]:
# using the priorized practice to aggregate GHG mitigation and on-farm costs
est_dom_cornfield = pd.read_csv(county_outputs_dir + 'county_dom_practice_cornfield.csv')
est_dom_cropland = pd.read_csv(county_outputs_dir + 'county_dom_practice_cropland.csv')

#------------------------
# aggregate GHG mitigation for current adoption in the US cornfields
mitigation_current = pd.DataFrame()
mitigation_current_se = pd.DataFrame()

corn_area = sum_county['corn_area']
adopt_rate_cc = sum_county['cc_ratio']
adopt_rate_nt = sum_county['nt_ratio']
# adopt_rate, farm_op_cc, farm_op_nt, lt_save_cc, lt_save_nt, corn_pr, soy_pr, co2_pr, ind_EF, re_rate
cty_sum = Total_mitigation(corn_area, est_dom_cornfield, adopt_rate_cc, adopt_rate_nt, -98, 57, 32, -29, 143, 359, 46, 0.011, 0.6) # mean margin
cty_sum_max = Total_mitigation(corn_area, est_dom_cornfield, adopt_rate_cc, adopt_rate_nt, -40, 95, 42, -12, 136, 314, 167, 0.011, 0.6) # max margin
cty_sum_min = Total_mitigation(corn_area, est_dom_cornfield, adopt_rate_cc, adopt_rate_nt, -206, 36, 26, -39, 170, 468, 27, 0.011, 0.6) # min margin

cty_sum = pd.concat(cty_sum, ignore_index=True)
cty_sum_max = pd.concat(cty_sum_max, ignore_index=True)
cty_sum_min = pd.concat(cty_sum_min, ignore_index=True)
cty_sum_all = pd.concat((cty_sum, cty_sum_max, cty_sum_min), ignore_index=True)
mitigation_current = pd.concat((mitigation_current, cty_sum_all), axis = 1, ignore_index=True)

mitigation_current = mitigation_current.transpose()

columns = ['cc_cty_sum', 'nt_cty_sum', 'int_dom_sum', 'eff_dom_sum']
targets = ['_soc', '_yld', '_ghg', '_fc', '_sb', '_mrg']
scenarios = ['', '_max', '_min']
mitigation_current.columns = [col + target + sce for sce in scenarios for col in columns for target in targets]
mitigation_current.insert(0, 'adoption rate', ['current'])
mitigation_current.to_csv(county_outputs_dir + 'Mitigation_potential_current_cornfield.csv', index=False)

corn_area = sum_county['corn_area']
adopt_rate_cc = sum_county['cc_ratio']
adopt_rate_nt = sum_county['nt_ratio']
# adopt_rate, farm_op_cc, farm_op_nt, lt_save_cc, lt_save_nt, corn_pr, soy_pr, co2_pr, ind_EF, re_rate
cty_sum = Total_mitigation_se(corn_area, est_dom_cornfield, adopt_rate_cc, adopt_rate_nt, -98, 57, 32, -29, 143, 359, 46, 0.011, 0.6) # mean margin
cty_sum_max = Total_mitigation_se(corn_area, est_dom_cornfield, adopt_rate_cc, adopt_rate_nt, -40, 95, 42, -12, 136, 314, 167, 0.011, 0.6) # max margin
cty_sum_min = Total_mitigation_se(corn_area, est_dom_cornfield, adopt_rate_cc, adopt_rate_nt, -206, 36, 26, -39, 170, 468, 27, 0.011, 0.6) # min margin

cty_sum = pd.concat(cty_sum, ignore_index=True)
cty_sum_max = pd.concat(cty_sum_max, ignore_index=True)
cty_sum_min = pd.concat(cty_sum_min, ignore_index=True)
cty_sum_all = pd.concat((cty_sum, cty_sum_max, cty_sum_min), ignore_index=True)
mitigation_current_se = pd.concat((mitigation_current_se, cty_sum_all), axis = 1, ignore_index=True)
mitigation_current_se = mitigation_current_se.transpose()

columns = ['cc_cty_sum', 'nt_cty_sum', 'int_dom_sum', 'eff_dom_sum']
targets = ['_soc', '_yld', '_ghg', '_fc', '_sb', '_mrg']
scenarios = ['', '_max', '_min']
mitigation_current_se.columns = [col + target + sce for sce in scenarios for col in columns for target in targets]
mitigation_current_se.insert(0, 'adoption rate', ['current'])
mitigation_current_se.to_csv(county_outputs_dir + 'Mitigation_potential_current_cornfield_se.csv', index=False)
#------------------------


#------------------------
# aggregate GHG mitigation for current adoption in the US croplands
mitigation_current = pd.DataFrame()
mitigation_current_se = pd.DataFrame()


# adopt_rate, farm_op_cc, farm_op_nt, lt_save_cc, lt_save_nt, corn_pr, soy_pr, co2_pr, ind_EF, re_rate
cty_sum = Total_mitigation(crop_area, est_dom_cropland, adopt_rate_cc, adopt_rate_nt, -98, 57, 32, -29, 143, 359, 46, 0.011, 0.6) # mean margin
cty_sum_max = Total_mitigation(crop_area, est_dom_cropland, adopt_rate_cc, adopt_rate_nt, -40, 95, 42, -12, 136, 314, 167, 0.011, 0.6) # max margin
cty_sum_min = Total_mitigation(crop_area, est_dom_cropland, adopt_rate_cc, adopt_rate_nt, -206, 36, 26, -39, 170, 468, 27, 0.011, 0.6) # min margin

cty_sum = pd.concat(cty_sum, ignore_index=True)
cty_sum_max = pd.concat(cty_sum_max, ignore_index=True)
cty_sum_min = pd.concat(cty_sum_min, ignore_index=True)
cty_sum_all = pd.concat((cty_sum, cty_sum_max, cty_sum_min), ignore_index=True)
mitigation_current = pd.concat((mitigation_current, cty_sum_all), axis = 1, ignore_index=True)

mitigation_current = mitigation_current.transpose()

columns = ['cc_cty_sum', 'nt_cty_sum', 'int_dom_sum', 'eff_dom_sum']
targets = ['_soc', '_yld', '_ghg', '_fc', '_sb', '_mrg']
scenarios = ['', '_max', '_min']
mitigation_current.columns = [col + target + sce for sce in scenarios for col in columns for target in targets]
mitigation_current.insert(0, 'adoption rate', ['current'])
mitigation_current.to_csv(county_outputs_dir + 'Mitigation_potential_current_cropland.csv', index=False)

crop_area = sum_county['crop_area']
adopt_rate_cc = sum_county['cc_ratio']
adopt_rate_nt = sum_county['nt_ratio']
# adopt_rate, farm_op_cc, farm_op_nt, lt_save_cc, lt_save_nt, corn_pr, soy_pr, co2_pr, ind_EF, re_rate
cty_sum = Total_mitigation_se(crop_area, est_dom_cropland, adopt_rate_cc, adopt_rate_nt, -98, 57, 32, -29, 143, 359, 46, 0.011, 0.6) # mean margin
cty_sum_max = Total_mitigation_se(crop_area, est_dom_cropland, adopt_rate_cc, adopt_rate_nt, -40, 95, 42, -12, 136, 314, 167, 0.011, 0.6) # max margin
cty_sum_min = Total_mitigation_se(crop_area, est_dom_cropland, adopt_rate_cc, adopt_rate_nt, -206, 36, 26, -39, 170, 468, 27, 0.011, 0.6) # min margin

cty_sum = pd.concat(cty_sum, ignore_index=True)
cty_sum_max = pd.concat(cty_sum_max, ignore_index=True)
cty_sum_min = pd.concat(cty_sum_min, ignore_index=True)
cty_sum_all = pd.concat((cty_sum, cty_sum_max, cty_sum_min), ignore_index=True)
mitigation_current_se = pd.concat((mitigation_current_se, cty_sum_all), axis = 1, ignore_index=True)
mitigation_current_se = mitigation_current_se.transpose()

columns = ['cc_cty_sum', 'nt_cty_sum', 'int_dom_sum', 'eff_dom_sum']
targets = ['_soc', '_yld', '_ghg', '_fc', '_sb', '_mrg']
scenarios = ['', '_max', '_min']
mitigation_current_se.columns = [col + target + sce for sce in scenarios for col in columns for target in targets]
mitigation_current_se.insert(0, 'adoption rate', ['current'])
mitigation_current_se.to_csv(county_outputs_dir + 'Mitigation_potential_current_cropland_se.csv', index=False)
#------------------------


#------------------------
# aggregate GHG mitigation for cornfields in the US
mitigation_cornfield = pd.DataFrame()
mitigation_cornfield_se = pd.DataFrame()
for (adopt_rate_cc, adopt_rate_nt) in zip([0.01,0.3,0.8,1],[0.01,0.31,0.52,1]):
    # adopt_rate, farm_op_cc, farm_op_nt, lt_save_cc, lt_save_nt, corn_pr, soy_pr, co2_pr, ind_EF, re_rate
    cty_sum = Total_mitigation(corn_area, est_dom_cornfield, adopt_rate_cc, adopt_rate_nt, -98, 57, 32, -29, 143, 359, 46, 0.011, 0.6) # mean margin
    cty_sum_max = Total_mitigation(corn_area, est_dom_cornfield, adopt_rate_cc, adopt_rate_nt, -40, 95, 42, -12, 136, 314, 167, 0.011, 0.6) # max margin
    cty_sum_min = Total_mitigation(corn_area, est_dom_cornfield, adopt_rate_cc, adopt_rate_nt, -206, 36, 26, -39, 170, 468, 27, 0.011, 0.6) # min margin

    cty_sum = pd.concat(cty_sum, ignore_index=True)
    cty_sum_max = pd.concat(cty_sum_max, ignore_index=True)
    cty_sum_min = pd.concat(cty_sum_min, ignore_index=True)
    cty_sum_all = pd.concat((cty_sum, cty_sum_max, cty_sum_min), ignore_index=True)
    mitigation_cornfield = pd.concat((mitigation_cornfield, cty_sum_all), axis = 1, ignore_index=True)

mitigation_cornfield = mitigation_cornfield.transpose()

columns = ['cc_cty_sum', 'nt_cty_sum', 'int_dom_sum', 'eff_dom_sum']
targets = ['_soc', '_yld', '_ghg', '_fc', '_sb', '_mrg']
scenarios = ['', '_max', '_min']
mitigation_cornfield.columns = [col + target + sce for sce in scenarios for col in columns for target in targets]
mitigation_cornfield.insert(0, 'adoption rate', ['1', 'lower limit', 'upper limit', '100'])
mitigation_cornfield.to_csv(county_outputs_dir + 'Mitigation_potential_cornfield.csv', index=False)

for (adopt_rate_cc, adopt_rate_nt) in zip([0.01,0.3,0.8,1],[0.01,0.31,0.52,1]):
#     corn_area = sum_county['corn_area'].fillna(0)
    # adopt_rate, farm_op_cc, farm_op_nt, lt_save_cc, lt_save_nt, corn_pr, soy_pr, co2_pr, ind_EF, re_rate
    cty_sum = Total_mitigation_se(corn_area, est_dom_cornfield, adopt_rate_cc, adopt_rate_nt, -98, 57, 32, -29, 143, 359, 46, 0.011, 0.6) # mean margin
    cty_sum_max = Total_mitigation_se(corn_area, est_dom_cornfield, adopt_rate_cc, adopt_rate_nt, -40, 95, 42, -12, 136, 314, 167, 0.011, 0.6) # max margin
    cty_sum_min = Total_mitigation_se(corn_area, est_dom_cornfield, adopt_rate_cc, adopt_rate_nt, -206, 36, 26, -39, 170, 468, 27, 0.011, 0.6) # min margin

    cty_sum = pd.concat(cty_sum, ignore_index=True)
    cty_sum_max = pd.concat(cty_sum_max, ignore_index=True)
    cty_sum_min = pd.concat(cty_sum_min, ignore_index=True)
    cty_sum_all = pd.concat((cty_sum, cty_sum_max, cty_sum_min), ignore_index=True)
    mitigation_cornfield_se = pd.concat((mitigation_cornfield_se, cty_sum_all), axis = 1, ignore_index=True)
mitigation_cornfield_se = mitigation_cornfield_se.transpose()

columns = ['cc_cty_sum', 'nt_cty_sum', 'int_dom_sum', 'eff_dom_sum']
targets = ['_soc', '_yld', '_ghg', '_fc', '_sb', '_mrg']
scenarios = ['', '_max', '_min']
mitigation_cornfield_se.columns = [col + target + sce for sce in scenarios for col in columns for target in targets]
mitigation_cornfield_se.insert(0, 'adoption rate', ['1', 'lower limit', 'upper limit', '100'])
mitigation_cornfield_se.to_csv(county_outputs_dir + 'Mitigation_potential_cornfield_se.csv', index=False)
#------------------------


#------------------------
# aggregate GHG mitigation for croplands in the US
mitigation_cropland = pd.DataFrame()
mitigation_cropland_se = pd.DataFrame()
for (adopt_rate_cc, adopt_rate_nt) in zip([0.01,0.3,0.8,1],[0.01,0.31,0.52,1]):
#     crop_area = sum_county['crop_area'].fillna(0)
    # adopt_rate, farm_op_cc, farm_op_nt, lt_save_cc, lt_save_nt, corn_pr, soy_pr, co2_pr, ind_EF, re_rate
    cty_sum = Total_mitigation(crop_area, est_dom_cropland, adopt_rate_cc, adopt_rate_nt, -98, 57, 32, -29, 143, 359, 46, 0.011, 0.6) # mean margin
    cty_sum_max = Total_mitigation(crop_area, est_dom_cropland, adopt_rate_cc, adopt_rate_nt, -40, 95, 42, -12, 136, 314, 167, 0.011, 0.6) # max margin
    cty_sum_min = Total_mitigation(crop_area, est_dom_cropland, adopt_rate_cc, adopt_rate_nt, -206, 36, 26, -39, 170, 468, 27, 0.011, 0.6) # min margin

    cty_sum = pd.concat(cty_sum, ignore_index=True)
    cty_sum_max = pd.concat(cty_sum_max, ignore_index=True)
    cty_sum_min = pd.concat(cty_sum_min, ignore_index=True)
    cty_sum_all = pd.concat((cty_sum, cty_sum_max, cty_sum_min), ignore_index=True)
    mitigation_cropland = pd.concat((mitigation_cropland, cty_sum_all), axis = 1, ignore_index=True)
mitigation_cropland = mitigation_cropland.transpose()

columns = ['cc_cty_sum', 'nt_cty_sum', 'int_dom_sum', 'eff_dom_sum']
targets = ['_soc', '_yld', '_ghg', '_fc', '_sb', '_mrg']
scenarios = ['', '_max', '_min']
mitigation_cropland.columns = [col + target + sce for sce in scenarios for col in columns for target in targets]
mitigation_cropland.insert(0, 'adoption rate', ['1', 'lower limit', 'upper limit', '100'])
mitigation_cropland.to_csv(county_outputs_dir + 'Mitigation_potential_cropland.csv', index=False)

for (adopt_rate_cc, adopt_rate_nt) in zip([0.01,0.3,0.8,1],[0.01,0.31,0.52,1]):
    crop_area = sum_county['crop_area'].fillna(0)
    # adopt_rate, farm_op_cc, farm_op_nt, lt_save_cc, lt_save_nt, corn_pr, soy_pr, co2_pr, ind_EF, re_rate
    cty_sum = Total_mitigation_se(crop_area, est_dom_cropland, adopt_rate_cc, adopt_rate_nt, -98, 57, 32, -29, 143, 359, 46, 0.011, 0.6) # mean margin
    cty_sum_max = Total_mitigation_se(crop_area, est_dom_cropland, adopt_rate_cc, adopt_rate_nt, -40, 95, 42, -12, 136, 314, 167, 0.011, 0.6) # max margin
    cty_sum_min = Total_mitigation_se(crop_area, est_dom_cropland, adopt_rate_cc, adopt_rate_nt, -206, 36, 26, -39, 170, 468, 27, 0.011, 0.6) # min margin

    cty_sum = pd.concat(cty_sum, ignore_index=True)
    cty_sum_max = pd.concat(cty_sum_max, ignore_index=True)
    cty_sum_min = pd.concat(cty_sum_min, ignore_index=True)
    cty_sum_all = pd.concat((cty_sum, cty_sum_max, cty_sum_min), ignore_index=True)
    mitigation_cropland_se = pd.concat((mitigation_cropland_se, cty_sum_all), axis = 1, ignore_index=True)
mitigation_cropland_se = mitigation_cropland_se.transpose()

columns = ['cc_cty_sum', 'nt_cty_sum', 'int_dom_sum', 'eff_dom_sum']
targets = ['_soc', '_yld', '_ghg', '_fc', '_sb', '_mrg']
scenarios = ['', '_max', '_min']
mitigation_cropland_se.columns = [col + target + sce for sce in scenarios for col in columns for target in targets]
mitigation_cropland_se.insert(0, 'adoption rate', ['1', 'lower limit', 'upper limit', '100'])
mitigation_cropland_se.to_csv(county_outputs_dir + 'Mitigation_potential_cropland_se.csv', index=False)